## Main Notebook for Benchmark Analysis - Training and Evaluation

- Introduction: this Jupyter Notebook reflects the quote `Garbage In, Garbage Out" (GIGO)`. Cheers! 
- `We made the mistake to build on augmented data`, whereas `we had to strat with the initial data` . . . and we conculded with bad scores . . . 
- If we had the time to repeat the process, the correct order would be to work on the inital data with the basic nlp preprocess, and then repeat the benchmark analsys for the augmented data and comapre the predicitions of the comeptition. 
    - Now, for descipline reasons we repeated the process for the initial data but we based on the benchmark analsysi of the augmented data as we had not time to make it in the correct order (intial data training process is in jupyter : `initial_training_process.ipynb`)
    - As benchmark analysis takes time to manually tune some parameters . . .
- And if we wanted to descipline in the benchmarak analsyis in depth we had to rerun for the raw data without any basic nlp preproces ...tehn for teh data with basic nlp preprosess and tehn for the augmented data with teh nlp preprocess as thsi way reflect the affect each change has in our trainign process. 
    - In other words, we has firstly apply benchmark analsysi for the raw data, tehn for the data with the basic nlp preprocess, and then for both the augmented and nlp preproess data and compare the results . . . 

- Due to the extremely low scores in the competition lifeboard (being extremely close to 0 (0.000..) based on our own synthetic data) we would like to start this jupyter via analysing some moral lessons.
- Before diving into the analysis of this Jupyter notebook, we would like to share some key moral lessons to help other junior developers and data scientists avoid encountering similar challenges. It took us some time to recognize these lessons, but thanks to this assignment, we have gained a deeper understanding of the complexities involved in time and memory constraints associated with specific parameters in models and vectorized methods like TF-IDF. Additionally, we have learned to approach augmented data with caution, as it might be entirely misleading. Unfortunately, we spent much time to "fake" data instead of beggining with the initial ones beforehand.
    - `Moral Lesson 1`: Fake data (in our case, synthetic or augmented) can be deceptive. Always verify early to ensure it aligns with the truth. Unfortunately, it took us much time to ensure it. We confirmed this when we observed the low scores obtained from the unlabeled data provided in the competition practice phase. We "trusted" the "fake" report metrics and we spent time to hyper tune our best model in the augmented dataset wrongly.
    - `Moral Lesson 2`: We spent significant time running computational pipelines while attempting to follow related work due to the unique nature of our problem, which is both a highly imbalanced problem (with some classes containing only one instance) and a multi-class classification challenge. More specifically, in a way, we tried to follow the training process used in the paper [CICLe: Conformal In-Context Learning for Largescale Multi-Class Food
Risk Classification](https://aclanthology.org/2024.findings-acl.459.pdf).
    - Pipeline: `stratissfy` (to ensure that the distribution of target classes in the training and testing) --> `TF-idf` (to prepare text data for input into a classifier model, it must first be vectorized) --> `logistic regression classifier with One-vs-All (OvA) method`.
        - Why in our local pc this trial was computational costly, and we fianlly used multinomial logistic regression to speed up. 
            - In our case, we began with an augmented dataset larger than that of the competition participants, applied TF-IDF vectorization with a high max_features parameter (limited due to resource constraints), and trained four logistic regression classifiers (hazard, product, hazard-category, and product-category) with numerous classes—making the process computationally intensive despite logistic regression's lightweight nature.
            - Computational Cost for Logistic Regression (OvA): Training 4 classifiers with `C=128,22,1022,10` classes involves a total complexity of `O(1182⋅n⋅d⋅iter)`, where `n` is the `number of samples`, `d the number of features`, and `iter` the optimization steps.
            - To reduce the dimensionality of input from TF-IDF, techniques like PCA (Principal Component Analysis) can be applied, as demonstrated in the latter part of this notebook. Alternatively, limiting the max_features parameter of TF-IDF, which we also implemented, is another effective approach.
            - To apply stratify, the presence of more data like synthetic data was necessary, as many classes had only one instance. In our case, we generated synthetic data using a synonym-based process, but unfortunately, it proved ineffective due to the "fake" nature of the data. An alternative approach could be using methods like SMOTE to address class imbalance. However, this can be computationally expensive when combined with TF-IDF; a practical trick is applying PCA to reduce the dimensionality of the TF-IDF vectors.
- If we had understood our "fake" data in advances via predicting the unlabeled data before applying hyperparameter tuning and trust the evaluation metrics, we would focus on the initial data only and we would re run the pipeline in the initial data trying to tune with the best way the model bearing in mind the time and memory limitations. 
    - We rerun the the whole process based on the **initila data** having only basi nlp-preprocess in the jupyetr notebook `initial_training_process.ipynb`, unfortunately we could not have time to tune the "best" model(s) in this jupyter duw to the moral lesson we aforementioned. 
    - Note : In the scope of benchmark analysis , it is crucial to run the pipeline for each change someone makes to have an overview about how an element affect the predictions. In our case how the dataset affect the prediction being the first element of the pipeline. 
        - Only, with this way the benchmark analysis based on strict discipline, but of course it is quite time consuming, as we have 4 individual classifiers (hazard, product, hazard-category, product-category), 2 datasets (augmented and intial data) and 2 Inputs (text , title).
--------
- `From this point and below begins the implementation process` : 
- This Jupyter Notebook contains the `benchmark analysis` based firstly on `"title"` as input and secondly on `"text"` as input. 
- Our aim is to detect the best model (`LogisticRegression`, `Random Forest` ans `X-Boost`) with the best Input "title" or "text".
- Then we will try to improve more  based on `automated hyperparameter tuning techniques` **only the model that was detected as the best one**.
    - Additionally, we developed `baseline models`, including a `majority` classifier and a `random` classifier, to establish `minimum performance benchmarks`. 
    - These baselines provide a reference point that our trained traditional and advanced classifiers are expected to surpass.
        - We want our tained traditional and advanced models to **outperform** the evaluations metrics of these baselines. 
        - In other words, we aim for our models to achieve predictive performance superior to random guessing or simply predicting the most frequent value.
---
> Evangelia P. Panourgia, Master Student in Data Science, AUEB <br />
> Department of Informatics, Athens University of Economics and Business <br />
> eva.panourgia@aueb.gr <br/><br/>


### Install Libraries / Setting the Scene

- `import os`: Provides functions for interacting with the operating system, such as file and directory management.
- `import pandas as pd`: A powerful library for data manipulation and analysis, particularly useful for handling tabular data (e.g., DataFrames).
- `import string`: Contains common string operations and constants like ASCII letters, digits, and punctuation.
- `import random`: Offers tools for generating random numbers, selecting random elements, and shuffling data.
- `import numpy as np`: A fundamental library for numerical computations in Python, including support for arrays, matrices, and mathematical operations.
- `from sklearn.metrics import f1_score, classification_report`:
  - `f1_score`: Calculates the F1 score, a weighted average of precision and recall.
  - `classification_report`: Generates a comprehensive report of classification metrics like precision, recall, and F1-score for all classes.
- `from sklearn.dummy import DummyClassifier`: A simple baseline classifier that makes predictions using simple rules (e.g., stratified, most frequent, etc.), useful for performance benchmarking.
- `from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, StratifiedShuffleSplit`:
  - `StratifiedKFold`: A cross-validation strategy that ensures each fold has a proportional representation of each class.
  - `train_test_split`: Splits data into training and testing subsets.
  - `cross_val_score`: Evaluates a model's performance using cross-validation.
  - `StratifiedShuffleSplit`: Creates train-test splits while maintaining class distribution.
- `from sklearn.linear_model import LogisticRegression`: Implements logistic regression, a linear model for binary or multi-class classification problems.
- `from sklearn.feature_extraction.text import TfidfVectorizer`: Converts text data into numerical representations using Term Frequency-Inverse Document Frequency (TF-IDF), a common text vectorization technique.
- `from sklearn.pipeline import Pipeline`: Facilitates creating and managing machine learning pipelines, enabling sequential application of transformations and model training.
- `from sklearn.utils.class_weight import compute_class_weight`: Computes weights for each class, often used to address class imbalance.
- `from sklearn.preprocessing import LabelEncoder`: Encodes target labels as integers, useful for transforming categorical labels into a numeric format.
- `from sklearn.multiclass import OneVsRestClassifier`: Implements the One-vs-Rest strategy for multi-class classification by training one binary classifier per class.
- `from sklearn.ensemble import StackingClassifier`: Combines multiple base classifiers in a stacked ensemble to improve predictive performance.
- `from sklearn.svm import SVC`: Implements a Support Vector Classifier (SVC) for binary or multi-class classification problems.
- `from xgboost import XGBClassifier`: A highly efficient and scalable implementation of gradient boosting designed for supervised learning tasks.
- `import joblib`: A library for saving and loading Python objects, such as trained machine learning models, in an efficient binary format.
- `from sklearn.ensemble import RandomForestClassifier`: Implements a Random Forest classifier, an ensemble learning method that constructs multiple decision trees and outputs the mode of their predictions.

In [1]:
!pip install  xgboost

### Setting the Scene 
- We will import all the needeed libraries.

In [1]:
import os
import pandas as pd
import string
import random
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import joblib  
from sklearn.ensemble import RandomForestClassifier

### Load Data 
- We will load the preprocessed data (`data/data_augmented__nlp_incidents_train.csv`) being pre-processed with `data augmented` (generation of synthetic data usage synonyms) and basic nlp preprocess (lowcase, stemming, lemma etc).
- Furthermore, we will load the unlabeleed data of the competition in ordeer to predict them (`data/incidents.csv`).
    - We applied the nlp-preprocess basic technique to the unlabelled data. However, we observed that this preprocessing step had minimal impact on the overall performance of the models, though it resulted in a slight improvement.

In [2]:
df_augmented= pd.read_csv('data/data_augmented__nlp_incidents_train.csv') # load data after data augmentation
testset_competition = pd.read_csv('data/incidents.csv', index_col=0) # load testing data (conception phase, unlabeled):

- Note: We loaded the transformed - preprocessed dataframe because in the main code we will use for `splitting the training and test sets of the argument data with stratisfy` in order to hold the analogous of classes, and if we had classes of 1 instance it would a problem.
    More specifically : 
    - What Stratification Does: `Stratification ensures that the proportions of classes in the training and test sets reflect the proportions in the original dataset`. This is especially useful when you have imbalanced classes.
    - The Problem with Single Instances: `If a class has only one instance, stratified splitting can't properly divide it across both the training and test sets while maintaining class proportions`. For instance, the stratified split might try to place the instance into both sets, which is impossible.

In [3]:
df_augmented = df_augmented[['title','text','hazard-category','product-category','hazard','product']]
df_augmented.head(3) # preview preproccessed data 

,title,text,hazard-category,product-category,hazard,product
0,recal notif fsis-024-94,case number 024-94 date open 07/01/1994 date c...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage
1,recal notif fsis-033-94,case number 033-94 date open 10/03/1994 date c...,biological,"meat, egg and dairy products",listeria spp,sausage
2,recal notif fsis-014-94,case number 014-94 date open 03/28/1994 date c...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices


In [5]:
testset_competition.head(3)# preview test data 

,year,month,day,country,title,text
0,1994,5,5,us,Recall Notification: FSIS-017-94,Case Number: 017-94 \n Date Opene...
1,1994,5,12,us,Recall Notification: FSIS-048-94,Case Number: 048-94 \n Date Opene...
2,1995,4,16,us,Recall Notification: FSIS-032-95,Case Number: 032-95 \n Date Opene...


## Baselines Models 
- Having `Baseline models` at our disposal, it is crucial for evaluating classification performance in multiclass imbalance settings because it provides reference points for how well your model is performing relative to simple baseline classifiers. The `Random Classifier` and `Majority Classifier` are commonly used as benchmarks for the following reasons:

### Random Classifier 
- A Random Classifier predicts class labels randomly, with **uniform** based on the distribution of classes. It sets a minimal baseline and helps understand:
- `Baseline Performance`: This represents the expected performance `without learning from the data`. `If a model performs worse than a random classifier, it indicates either issues in the model or unsuitable features`.
- `Chance Levels`: It shows what performance you'd `get by chance alone`, especially useful for imbalanced datasets where metrics like accuracy can be misleading.


### Majority Classifier

- A Majority Classifier always **predicts the majority class** (`the class with the highest frequency in the training data`). 
- It helps understand:
    - `Handling Imbalance`: In multiclass imbalanced datasets, accuracy can be dominated by the majority class. The majority classifier provides a baseline to compare how well your model captures minority classes.
    - `Baseline of Naïve Solutions`: The majority classifier reflects the simplest possible rule for prediction. If a model's performance is close to that of a majority classifier, it suggests the model is failing to generalize or adapt to the minority classes.
    - `Focus on Class Imbalance`: Metrics like weighted accuracy, balanced accuracy, or macro-F1 score should be significantly better than those achieved by the majority classifier to indicate that a model is addressing imbalance effectively.
- Note in the following code cell we implement the code for Random and Majority Classifier, **note** in order to have a high level of "logic" we added the split steps of trainingtest set, but in our case of random and majority classifier it is useless as **both of them are not affected from the input**, in other words they don't learn from the input data they are based on statistics methos (rendomness, mode).
    - Hoever, this "skeleton" is useful for the reamaining algorythms to buils in (both traditional and advanced) 
- More specifically, 
    - Random Classifier  Effect of X: The X values (features) **do not influence the random classifier's predictions**. It does not learn from the data in the feature column. Its predictions are purely random, so changing X will not alter its performance.
    - Majority Classifier Effect of X: The feature column X is ignored by the majority classifier, as it does not use features for prediction. Instead, it looks only at the distribution of y in the training data.

### Regarding the Implementation 
- The `DummyClassifier in scikit-learn` is a baseline model designed to evaluate classification algorithms by comparing them against simplistic strategies. These strategies provide minimal logic to make predictions and are often used as benchmarks to understand how well a more complex model performs.
    - `strategy="uniform"` (for Random Classifier): 
        - Predicts a class randomly and uniformly across all possible classes.
        - Each class has an equal probability of being selected, irrespective of the class distribution in the training data.
        - Use Case: Ideal for scenarios where you want to simulate random guessing.
    - `strategy="most_frequent"` (for Majoriry Classification)
        - Always predicts the most frequent class observed in the training data.
        - Ignores the input features entirely and focuses only on the training set's class distribution.
        - Use Case: Useful for understanding how well a naive baseline would perform if you simply predicted the majority class.

In [6]:
def evaluate_baselines(dataframe, feature_column):
    """
    Function to evaluate random and majority classifiers on a given dataframe.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.
    """
    np.random.seed(42)  # For reproducibility

    # Train-test split with optional stratification
    trainset, testset = train_test_split(
        dataframe, 
        test_size=0.2, 
        random_state=2024, 
        # "skeleton" for the main algo here add stratisfy to hold proportion of classes 
    )
   
    # Random and Majority classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Evaluating for label: {label}")

        # Features and target
        X_train = trainset[feature_column]
        y_train = trainset[label]
        X_test = testset[feature_column]
        y_test = testset[label]

        # Random Classifier
        random_clf = DummyClassifier(strategy="uniform", random_state=2024)
        random_clf.fit(X_train, y_train) # it is uselless X stimulate the logic of a real algo. 
        testset['predictions-random-' + label] = random_clf.predict(X_test)

        # Majority Classifier
        majority_clf = DummyClassifier(strategy="most_frequent")
        majority_clf.fit(X_train, y_train)# it is uselless X stimulate the logic of a real algo. 
        testset['predictions-majority-' + label] = majority_clf.predict(X_test)

        # Compute F1 scores
        random_f1 = f1_score(y_test, testset['predictions-random-' + label], average='macro', zero_division=0)
        majority_f1 = f1_score(y_test, testset['predictions-majority-' + label], average='macro', zero_division=0)

        print(f"F1 Score for Random Classifier ({label}): {random_f1:.3f}")
        print(f"F1 Score for Majority Classifier ({label}): {majority_f1:.3f}")

        # Generate and save classification reports
        os.makedirs('reports/random', exist_ok=True)
        os.makedirs('reports/majority', exist_ok=True)

        random_report = classification_report(y_test, testset['predictions-random-' + label], zero_division=0)
        majority_report = classification_report(y_test, testset['predictions-majority-' + label], zero_division=0)

        with open(f'reports/random/random_classifier_report_{label}.txt', 'w') as random_file:
            random_file.write(f"Classification Report for Random Classifier ({label}):\n")
            random_file.write(random_report)

        with open(f'reports/majority/majority_classifier_report_{label}.txt', 'w') as majority_file:
            majority_file.write(f"Classification Report for Majority Classifier ({label}):\n")
            majority_file.write(majority_report)
        
        
    
    # Custom metric score calculation
    def compute_score(hazards_true, products_true, hazards_pred, products_pred):
        """
        Custom scoring function to compute the macro F1 score for hazards and products.
        
        Args:
            hazards_true: Ground truth labels for hazards.
            products_true: Ground truth labels for products.
            hazards_pred: Predicted labels for hazards.
            products_pred: Predicted labels for products.
        
        Returns:
            A float representing the combined macro F1 score.
        """
        f1_hazards = f1_score(hazards_true, hazards_pred, average='macro', zero_division=0)
        f1_products = f1_score(
            products_true[hazards_pred == hazards_true],
            products_pred[hazards_pred == hazards_true],
            average='macro', 
            zero_division=0
        )
        return (f1_hazards + f1_products) / 2.

    # Example of calculating scores for Sub-Tasks (if needed):
    # Uncomment the following lines to compute scores for tasks
    print(f"Score Sub-Task 1 - Random Classifier: {compute_score(testset['hazard-category'], testset['product-category'], testset['predictions-random-hazard-category'], testset['predictions-random-product-category']):.3f}")
    print(f"Score Sub-Task 2 - Random Classifier: {compute_score(testset['hazard'], testset['product'], testset['predictions-random-hazard'], testset['predictions-random-product-category']):.3f}")
    print(f"Score Sub-Task 1 - Majority Classifier: {compute_score(testset['hazard-category'], testset['product-category'], testset['predictions-majority-hazard-category'], testset['predictions-majority-product-category']):.3f}")
    print(f"Score Sub-Task 2 - Majority Classifier: {compute_score(testset['hazard'], testset['product'], testset['predictions-majority-hazard'], testset['predictions-majority-product']):.3f}")

# Call the function with the required dataframe (e.g., df_augmented or any other dataframe)
evaluate_baselines(df_augmented, feature_column='text') # we pass only text we dobn't re-run for "title" as these classifiers are not affected by the Input X. tahy are based on statistics (random, mode)

Evaluating for label: hazard-category
F1 Score for Random Classifier (hazard-category): 0.076
F1 Score for Majority Classifier (hazard-category): 0.047
Evaluating for label: product-category


F1 Score for Random Classifier (product-category): 0.034
F1 Score for Majority Classifier (product-category): 0.019
Evaluating for label: hazard
F1 Score for Random Classifier (hazard): 0.005
F1 Score for Majority Classifier (hazard): 0.001
Evaluating for label: product
F1 Score for Random Classifier (product): 0.000
F1 Score for Majority Classifier (product): 0.000
Score Sub-Task 1 - Random Classifier: 0.057
Score Sub-Task 2 - Random Classifier: 0.003
Score Sub-Task 1 - Majority Classifier: 0.031
Score Sub-Task 2 - Majority Classifier: 0.001


- Evaluating for label: hazard-category
    - F1 Score for Random Classifier (hazard-category): 0.076
    - F1 Score for Majority Classifier (hazard-category): 0.047
- Evaluating for label: product-category
    - F1 Score for Random Classifier (product-category): 0.034
    - F1 Score for Majority Classifier (product-category): 0.019
- Evaluating for label: hazard
    - F1 Score for Random Classifier (hazard): 0.005
    - F1 Score for Majority Classifier (hazard): 0.001
- Evaluating for label: product
    - F1 Score for Random Classifier (product): 0.000
    - F1 Score for Majority Classifier (product): 0.000

- Score Sub-Task 1 - Random Classifier: 0.057
- Score Sub-Task 2 - Random Classifier: 0.003
- Score Sub-Task 1 - Majority Classifier: 0.031
- Score Sub-Task 2 - Majority Classifier: 0.001

###  Results and Observations (For reproduction we have saved in markdown the scores of baseline models)
- Label: `hazard-category`
    - Random Classifier F1: 0.076
    - Majority Classifier F1: 0.047
- Performance is slightly better for the Random Classifier, but both are low, indicating the dataset is likely imbalanced, and random guessing doesn't align well with true labels.

- Label: `product-category`
    - Random Classifier F1: 0.034
    - Majority Classifier F1: 0.019
- Performance drops further here. It suggests more complexity or higher imbalance in this label.

- Label: `hazard`
    - Random Classifier F1: 0.005
    - Majority Classifier F1: 0.001
    - Both scores are extremely low, possibly due to:
        - Large number of classes.
        - Sparse distribution of classes.
        - Poor representation of these classes in the Random Classifier's uniform predictions or Majority Classifier's mode.

- Label: `product`
    - Random Classifier F1: 0.000
    - Majority Classifier F1: 0.000
    - Both classifiers completely fail to capture meaningful patterns for this label. This could suggest extreme imbalance or lack of meaningful correlation in the dataset.

- `Sub-Tasks`
    - `Score Sub-Task 1: hazard-category & product-category`
        - Random Classifier Score: 0.057
        - Majority Classifier Score: 0.031
        - Indicates the overall performance when combining macro F1 scores for hazard-category and product-category. Random guessing outperforms predicting the most frequent class, but both are weak.
    - `Score Sub-Task 2: hazard & product`
        - Random Classifier Score: 0.003
        - Majority Classifier Score: 0.001
        - Reflects the severe challenge for these labels. The performance is near zero, affirming the labels require more sophisticated approaches.

-  `Conclusions` : 
- Baseline Analysis:

    - **The poor F1 scores highlight the challenging nature of the task and dataset**.
    - These results provide a benchmark to evaluate future models. Any model achieving significantly higher F1 scores would demonstrate effective learning.

- Dataset Imbalance:

    - The low performance of the Majority Classifier indicates severe class imbalance across all labels.
    - Future models should address this using strategies like stratified sampling, oversampling, or weighted loss functions.

- Complexity of Labels:

    - The complexity increases from hazard-category and product-category to hazard and product, as reflected in the declining F1 scores.

- Actionable Insights:

    - Preprocessing: Investigate the class distributions and apply balancing techniques.
    - Feature Engineering: Consider enhancing the feature column (e.g., using embeddings).
    - Advanced Models: Apply models capable of handling imbalance, such as `tree-based methods`, `ensemble models`, or neural networks.


## Traditional and Advanced Algorythms


## Strategy for Model Selection and Evaluation

### Overview
Our approach systematically evaluates three machine learning algorithms across two input types: **"title"** and **"text."** The primary objective is to identify the `best-performing model` using evaluation metrics, with a focus on the custom competition evaluation metric.

Given time and memory constraints, specific parameter values were set for each algorithm following an initial manual investigation. For each algorithm, we:  
- **a)** Adjusted hyperparameters to align with the nature of the problem (multi-class classification).  
- **b)** Focused on manually adapting hyperparameters to improve the custom metric. This was our main goal during the benchmark adjustment of parameters.   
- **c)** Some hyperparameters were adjusted to align with memory and time constraints. For instance, the max_features parameter in TF-IDF was fine-tuned. Additionally, in logistic regression, we opted for the multinomial approach instead of the One-vs-All classification strategy to optimize resource utilization.

While some degree of overfitting to specific classes is expected—particularly for classes with low support due to data imbalance—this trade-off is carefully managed. This is evident in cases where significant differences between precision and recall are observed, especially in underrepresented classes.

The benchmark analysis provides a comprehensive overview of model performance for both input types (**"title"** and **"text"**) and guides decisions for **further optimization.** Future steps may include:  
- Systematic hyperparameter tuning (e.g., grid search or Optuna).  
- Dimensionality reduction techniques like PCA to enhance performance.  

### Strategy we followed in high level
- For each algorythm we manually adapt the parameters of the model and tf-idf (being our preprocess vectorization step). These adaptation of parameters based on two main diamensions: 
  - `a)` We tailored each algorithm to align with the **nature of the problem**. For example, in Logistic Regression, we addressed the multiclass classification challenge by setting the parameter `multi_class='multinomial'`. Similarly, for XGBoost, we adapted it to the multiclass problem using the objective `multi:softmax` and specifying `eval_metric='mlogloss'` to evaluate performance effectively.
  - `b)` We focused mostly on the custom score of the competition as the goal of the competition is to increase it. 
  - `c)` We further analyzed the average macro F1 scores (a key metric for the competition) alongside the individual per-class F1 scores derived from the classification reports. This deeper analysis aimed to identify specific classes where the model struggled with predictions. While valuable for understanding model performance, this detailed evaluation is less impactful at this stage, primarily due to class imbalance. Classes with low support tend to exhibit significant disparities between precision and recall. `However, in future work, this insight could be leveraged to regroup poorly predicted classes into a new category e.g. Other, potentially improving the custom competition score. And we could apply it only for the hazard classes as hazard affect more the competition score that the product ones.` Due to time constraints, our primary focus remained on the evaluation described in section b).
  - `Preprocess Strategy Vectorization`: For tf-idf, to ensure comparability across the results of different algorithms, we maintained a consistent vectorization strategy. The only parameter we manually adjusted was max_features. However, we observed that varying this parameter, both for the title and text features, did not lead to an improvement in the competition score. the remaining parameters we used maintained constant (`(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5,`)
    - **Note** : the adaptation of the parameter `max_feature` has more meaning for the `text` input, but in our case we did not observe improvement in the score of the competition. 
      - So, we simple re-run the same models via changing only the input that is "title" and "text" of the models.
  - **Important Note**: Some parameters had to be adapted properlydue to time and memory limitations (e.g. OvA high complexity rejecte for practical reasons).
- Note all benchmarak analysis algorythms were evaluated mostly to competitino evaliuation score, in orer to compare with the same thing in order to be more fair the comarison.

### Why we  selected these algorythms in our benchmark analysis 
- The answer is because the these algorithms are great for benchmark analysis as **Logistic Regression models** `linear relationships`, **Random Forest** handles `nonlinear patterns through decision tree ensembles`, and **XGBoost** captures **complex** `nonlinear relationships using sequential, gradient-boosted trees for high predictive accuracy`.
- In more depth:
  - Key Difference Between Logistic Regression, Random Forest, and XGBoost:
    - `Logistic Regression` is a linear model for binary classification, offering simplicity and interpretability.
    - `Random Forest` is an ensemble of decision trees, focusing on reducing overfitting and handling complex, nonlinear relationships.
    - `XGBoost (Extreme Gradient Boosting)` is a gradient-boosting algorithm that builds trees sequentially, optimizing for speed and accuracy, and often outperforming Random Forest in predictive performance, especially on structured/tabular data.
  - For future work, if Random Forest and XGBoost yield the best results, we could enhance performance further by combining them using a `stack ensemble technique`; however, due to time constraints, we were unable to explore or compare this approach.

### Step-by-Step Strategy

#### 1. Initial Model Evaluation with "Title" Input
- **Algorithms Tested**:
  - `Logistic Regression`
  - `Random Forest`
  - `XGBoost`
- **Parameter Setting**:
  - Parameters for each algorithm are manually tuned based on preliminary analysis to balance performance and computational efficiency.
- **Evaluation**:
  - Models are assessed using:
    - Standard evaluation metrics (e.g., accuracy, precision, recall, F1-score).
    - A custom evaluation metric provided by the competition.
- **Objective**:
  - Identify the most `promising algorithm` based on "title" input.

#### 2. Evaluation with "Text" Input
- **Algorithms Tested**:
  - Logistic Regression
  - Random Forest
  - XGBoost
- **Evaluation**:
  - The same evaluation metrics and competition-specific metric are applied as in the "title" input analysis.
- **Objective**:
  - identify the most `promising algorithm` based on  "text" input.

#### 3. Comparison of Best Models
- The top-performing models from the **"title"** and **"text"** input evaluations are compared.
- **Selection**:
  - Based on their performance across all metrics, the superior model is selected.

#### 4. Optimization of the Final Model
- The chosen algorithm undergoes parameter optimization to refine its performance.
- **Constraints**:
  - Due to time limitations, cross-validation (e.g., K-fold validation) will applied one time (with K = 3) during hyperparameter tuning process bur for few hyperparameters combinations due to needed time needs.
  - Instead, a streamlined validation approach is used to ensure efficient optimization without excessive computational overhead.

#### 5. Baseline Comparison
- Throughout the process, model performance is benchmarked against baseline models:
  - **Random Prediction**: A model that predicts randomly.
  - **Majority Class Prediction**: A model that always predicts the most frequent class.
- **Objective**:
  - Provide context for evaluating the added value of the trained algorithms. We want our trained models both traditionals and advanced to outperforms the baseline models.

### Summary
This structured methodology ensures a thorough evaluation of multiple algorithms across different input types, with a focus on balancing computational efficiency and performance mostly of the competition. By the end of this process, the goal is to identify the best-performing algorithm with the best input title or text and optimize it for deployment within the constraints of time and resources.

### Part A. Benchmark Analysis Title


- Firstly,we will include `the custom evaluation metric` provided by the competition page.
    - It will be used for as evaluation part of all algorythms.
- The function `compute_score` is taht used in the scope of the competition.

In [126]:
## Helping function for the calculation of the custom evaluation for the subtasks.
def compute_score(hazards_true, products_true, hazards_pred, products_pred):
    """
    Compute a custom F1 score that considers hazards and products together.
    """
    # Reset indices to ensure alignment
    hazards_true = hazards_true.reset_index(drop=True)
    products_true = products_true.reset_index(drop=True)
    hazards_pred = pd.Series(hazards_pred).reset_index(drop=True)
    products_pred = pd.Series(products_pred).reset_index(drop=True)

    # Compute F1 for hazards
    f1_hazards = f1_score(hazards_true, hazards_pred, average='macro', zero_division=0)

    # Compute F1 for products, only where hazards predictions match ground truth
    mask = hazards_pred == hazards_true
    f1_products = f1_score(
        products_true[mask],
        products_pred[mask],
        average='macro',
        zero_division=0
    )

    # Return the combined metric
    return (f1_hazards + f1_products) / 2.

- With this measure, we base our rankings predominantly on the predictions for the hazard classes. Intuitively, this means that a submission with both hazards_pred and products_pred completely right will score 1.0, a submission with hazards_pred completely right and products_pred completely wrong will score 0.5, and any submission with hazards_pred completely wrong will score 0.0 independently of the value of products_pred.

# Logistic Regression TF-IDF Title 

### High-Level Explanation of `train_log_regression_classifiers`
### Objective
- Train multinomial logistic regression classifiers for four labels and compute custom metrics to evaluate performance.
### Inputs
- **`dataframe`**: Dataset containing features and target labels.
- **`feature_column`**: The column in the dataframe to be used for feature extraction.
### Outputs
- **`classifiers`**: A dictionary of trained logistic regression classifiers for each label.
- **`vectorizers`**: A dictionary of TF-IDF vectorizers used for feature extraction.
- **`custom_metrics`**: A dictionary of custom evaluation scores for subtasks on test data.
---
### Key Steps
1. **Initialization**:
   - Set a random seed for reproducibility.
   - Prepare dictionaries to store classifiers, vectorizers, and custom metrics, useful in order to predict the ullaabeled data if this model verified as the best one.
2. **Label-Specific Training**:
   - Iterate over four labels: 
     - `hazard-category`, 
     - `product-category`, 
     - `hazard`, 
     - `product`.
   - For each label:
     - Perform a stratified train-test split to maintain class distribution.
     - Use `TfidfVectorizer` to extract character-based n-gram features (2-5) with a maximum of 5000 features.
     - Train a multinomial logistic regression classifier with a maximum of 100 iterations.
     - Evaluate the model using F1 score and generate a classification report.
     - Save the trained classifier, vectorizer, and test data.
3. **Custom Metric Calculation**:
   - Compute task-specific metrics for two subtasks:
     - **Subtask 1**: Evaluate the relationship between `hazard-category` and `product-category`.
     - **Subtask 2**: Evaluate the relationship between `hazard` and `product`.
   - Combine F1 scores for hazards and products to compute the final metric for each subtask.
4. **Logging and Output**:
   - Print F1 scores for each label and custom metric scores for subtasks.
   - Save classification reports to a dedicated directory.
   - Return the trained classifiers, vectorizers, and custom metrics.
---
### Key Techniques
- **TF-IDF Vectorization**: Convert text into feature vectors using character n-grams.
- **Logistic Regression Classifier**: Train multinomial classifiers with limited iterations for resourses reasons (memory-time).
- **Custom Metric Calculation**: Evaluate subtasks by combining F1 scores for hazards and products.
This function facilitates multi-label text classification and provides task-specific evaluations with a focus on hazards and products.

In [13]:
def train_log_regression_classifiers(dataframe, feature_column):
    """
    Train multinomial logistic regression classifiers for four labels and calculate custom metrics on test data.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.

    Returns:
        classifiers: A dictionary containing trained classifiers for each label.
        vectorizers: A dictionary containing TF-IDF vectorizers for each label.
        custom_metrics: A dictionary containing the custom metric score for each pair of labels on test data.
    """
    np.random.seed(42)  # For reproducibility

    classifiers = {}  # Dictionary to store the trained classifiers
    vectorizers = {}  # Dictionary to store the TF-IDF vectorizers
    custom_metrics = {}  # Dictionary to store custom metric scores

    # Dictionaries to store test data for each category
    test_data = {}

    # Train classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Training classifier for label: {label}")

        # Train-test split with stratification based on the current label
        trainset, testset = train_test_split(
            dataframe,
            test_size=0.2,
            random_state=2024,
            stratify=dataframe[label] # mentain proportion of classes distribution
        )

        # Extract train and test features
        X_train = trainset[feature_column]
        X_test = testset[feature_column]

        # Target
        y_train = trainset[label]
        y_test = testset[label]

        # Define TfidfVectorizer for the current label
        vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5, max_features=5000) # limit to 5000 for resourse reasons speed up.
        #TODO space reduction strategy?
        vectorizers[label] = vectorizer

        # Transform features using the label-specific vectorizer
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Define and train Logistic Regression classifier
        classifier = LogisticRegression(max_iter=100, random_state=2024, multi_class='multinomial') # limit to max iter and use multinomial instead of OvA
        classifier.fit(X_train_tfidf, y_train)

        # Store the trained classifier
        classifiers[label] = classifier

        # Store test data
        test_data[label] = {
            'X_test_tfidf': X_test_tfidf,
            'y_test': y_test
        }

        # Predict and evaluate
        predictions = classifier.predict(X_test_tfidf)
        logreg_f1 = f1_score(y_test, predictions, average='macro', zero_division=0)
        print(f"F1 Score for {label}: {logreg_f1:.3f}")

        # Generate classification report
        report = classification_report(y_test, predictions, zero_division=0)
        print(report)

        # Save the report
        os.makedirs('reports/logreg', exist_ok=True)
        with open(f'reports/logreg/logreg_classifier_report_{label}_{feature_column}.txt', 'w') as logreg_file:
            logreg_file.write(f"Classification Report for {label}_{feature_column}:\n")
            logreg_file.write(report)
            logreg_file.write(f"F1 Score: {logreg_f1:.3f}\n")

    # Compute the custom metric for hazards and products using test data only
    custom_metrics['subtask_1'] = compute_score(
    test_data['hazard-category']['y_test'],
    test_data['product-category']['y_test'],
    classifiers['hazard-category'].predict(test_data['hazard-category']['X_test_tfidf']),
    classifiers['product-category'].predict(test_data['product-category']['X_test_tfidf'])
    )

    custom_metrics['subtask_2'] = compute_score(
    test_data['hazard']['y_test'],
    test_data['product']['y_test'],
    classifiers['hazard'].predict(test_data['hazard']['X_test_tfidf']),
    classifiers['product'].predict(test_data['product']['X_test_tfidf'])
    )


    print(f"Custom Metric for Subtask 1 (Test Data): {custom_metrics['subtask_1']:.3f}")
    print(f"Custom Metric for Subtask 2 (Test Data): {custom_metrics['subtask_2']:.3f}")

    return classifiers, vectorizers, custom_metrics #  return 4 trained classifiers {}, 4 vectorized - tf-idf {}, custom metrics {}

- Note, manually we tried seperate values for `TfidfVectorizer` for the parameter `max_features`, note, the default one in combiantion with having in Logistic Regression `max_iters` 1000, we observed that it was quite low. 
- So, we concluded to define TfidfVectorizer(max_features= 5000) and LogisticRegression(max_iters= 100)
- This manually experimentation was crucial due to limited local recourses.
- Furthermore, manually we tried instead of `multi_class='multinomial` in Logisticregression to use `one-vs-all` strategy, but it was computationally costy, so we concluded to `multinomial` one.
- Having trained our first traditional model, let's analyse the produced evaluation metrics. 
- We adapted the parameters via giving mostly emphasis on competition score to be improved. Of course the professioanl way was to use a systematic method like greed search or an MLop tool like optuna so taht the hyperparamter tuning being automatically, but due to time limitations we could not apply it.
- let's observe the evaluations metrics.

In [8]:
classifiers, vectorizers, custom_metrics = train_log_regression_classifiers(df_augmented, 'title')
print("Custom Metric Scores on Test Data:")
print(custom_metrics)

Training classifier for label: hazard-category


/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for hazard-category: 0.683
                                precision    recall  f1-score   support

                     allergens       0.82      0.93      0.87       863
                    biological       0.84      0.92      0.88       771
                      chemical       0.80      0.83      0.82       329
food additives and flavourings       1.00      0.21      0.35        14
                foreign bodies       0.86      0.75      0.80       253
                         fraud       0.84      0.71      0.77       280
                     migration       1.00      0.93      0.96        29
          organoleptic aspects       1.00      0.23      0.38        39
                  other hazard       0.92      0.46      0.61       103
              packaging defect       0.83      0.26      0.39        39

                      accuracy                           0.83      2720
                     macro avg       0.89      0.62      0.68      2720
                  weighted

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for product-category: 0.688
                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.95      0.50      0.66        38
                      cereals and bakery products       0.65      0.70      0.67       252
     cocoa and cocoa preparations, coffee and tea       0.77      0.71      0.74        99
                                    confectionery       0.81      0.37      0.51        91
dietetic foods, food supplements, fortified foods       0.79      0.78      0.78        95
                                    fats and oils       1.00      0.62      0.77        32
                                   feed materials       1.00      0.73      0.84        11
                   food additives and flavourings       1.00      0.70      0.82        10
                           food contact materials       1.00      0.89      0.94        35
                            fruits and vegetables   

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for hazard: 0.476
                                                   precision    recall  f1-score   support

                                        Aflatoxin       0.00      0.00      0.00         5
                                   abnormal smell       1.00      0.43      0.60         7
                                  alcohol content       0.82      0.90      0.86        10
                                        alkaloids       1.00      0.67      0.80         9
                                        allergens       0.00      0.00      0.00         7
                                           almond       0.60      0.48      0.54        31
             altered organoleptic characteristics       1.00      0.90      0.95        10
                                        amygdalin       0.62      0.89      0.73         9
                           antibiotics, vet drugs       0.00      0.00      0.00         6
                                    bacillus spp.       1.00  

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for product: 0.344
                                                                        precision    recall  f1-score   support

                                                Catfishes (freshwater)       0.40      0.67      0.50         3
                                                       Dried pork meat       0.00      0.00      0.00         1
                                                 Fishes not identified       0.17      0.44      0.24         9
                                                    Groupers (generic)       0.00      0.00      0.00         1
                                              Not classified pork meat       1.00      0.75      0.86         4
                                            Pangas catfishes (generic)       0.00      0.00      0.00         2
                                   Precooked cooked pork meat products       0.00      0.00      0.00         6
                                    Torpedo-shaped catfishes (generic)     

- **Importan Note**: At this cell we will explain the way in which we interpret the `evaluation report` in depth for the individual classifier `hazard-category`, due to time limitatiosn we will not repeat it for all models. 
  - We will directly compare the models and conclude to the best one based on custom evaluation score mostly. In other words, in the following evaluations of the other models we will pay attention more to the final competition sub tasks custmom evalautions, and to macro avg f1-score (being part of the competition custom score), as our aim is to find the best model. Of course, internally, we will quick observe the evalaution report as observing for each classes presion recall (or f1-score) we can understand if our models has `overfit` for spesific classes. However, the in-depth evaluation has more sense in our best model as at this stage we want a quich overview of wich model has a better potential to "win" the competition. 
    - `Note` : If we wanted to perfmorm  high score in the competition and we had more time at our dismposal we would try to group the classes with low performance to a new category "Other". 
- General comment : `Utilize metrics like **macro average F1 score** to emphasize balanced performance across all classes`. This is the reason why we focus more in this score and in competition custom score.
  - `The macro-averaged F1-score is particularly useful for datasets with class imbalances, as it evaluates how well the model performs across all classes without weighting them by their prevalence in the dataset.`

Notes for evaluation of `Logistic Regression for hazard-category` (input `title`): 
   - Firstly, The Logistic Regression model outperforms both the majority and random classifier baselines, which is a positive outcome. 
        - If the model’s evaluation metrics were close to those of a random classifier, it would indicate that the training process was ineffective, as the model would essentially be making predictions at random. 
        - Similarly, if the model’s evaluation metrics were close to those of the majority classifier, it would suggest that the model requires further optimization, as it primarily predicts the most frequent class instead of learning meaningful patterns from the data. 
   - Analysis of Evaluation Report : 
   ### General Observations for the first evaluation reports of `hazard-category` similar we interpret the remaining one for the other individual classfiiers:
   - **Overall Accuracy:** The model achieves an accuracy of **83%**, which is decent but insufficient as a standalone metric, especially for imbalanced datasets.
   - **Macro Avg (F1 Score):** The macro average F1 score is **0.68**, highlighting significant performance variation across categories. This is much lower than the weighted F1 score of **0.82**, suggesting better performance on frequently occurring categories.
   - **Weighted Avg (F1 Score):** The weighted F1 score is **0.82**, as it accounts for the class distribution, showing the model is biased toward the majority classes.

   ### Class-wise Performance:
   ### Strong Performers:
   - **Allergens (F1: 0.87):** High precision (**0.82**) and recall (**0.93**), indicating consistent performance.
   - **Biological (F1: 0.88):** Strong performance with balanced precision and recall.
   - **Chemical (F1: 0.82):** Good overall performance.
   - **Migration (F1: 0.96):** Excellent results, though support is very low (**29 samples**), raising concerns about generalizability.

   ### Weak Performers for Hazard-Category:
   - **Food Additives and Flavourings (F1: 0.35):** Perfect precision (**1.00**) but extremely poor recall (**0.21**), indicating very conservative predictions.
   - **Organoleptic Aspects (F1: 0.38):** Similar to the above, precision is high but recall is very low.
   - **Packaging Defect (F1: 0.39):** Poor recall (**0.26**) suggests difficulty identifying these cases.
   - **Other Hazard (F1: 0.61):** Moderate performance with low recall (**0.46**).
    - `note`: in the score of "winning" the competition these classes could be grouped to one and re-run in order to check if we will have better results.   
   ### Overfitting Indicators:
   - **Precision-Recall Disparity:** The significant gap between **macro average precision (0.89)** and **macro average recall (0.62)** suggests potential overfitting. The model confidently predicts a few dominant classes but fails to generalize to minority classes.
   - **Low Recall for Minority Classes:** For low-support categories like "food additives and flavourings" and "organoleptic aspects," the model achieves perfect precision but nearly zero recall, indicating memorization rather than generalization.

   ### Model's Strengths and Weaknesses:
   ### Strengths:
   - Excellent performance on majority classes like allergens and biological hazards.
   - High precision in predicting categories with substantial support.
  - Note similarly we interpret the remaining classification reports. 
   ## `Competition Metrics interpretation` 

  - `Subtask 1 (ST1): ~0.689`

  - This subtask involves text classification for food hazard prediction, which focuses on predicting the type of hazard and product.
A score of ~0.689 indicates that the classifier performs reasonably well in predicting hazards and products, but there is still room for improvement. Since the compute_score function emphasizes hazards (f1_hazards) more heavily, this score suggests the hazard predictions were better than the product predictions but not perfect.

  - `Subtask 2 (ST2): ~0.425`

  - This subtask involves food hazard and product "vector" detection, which focuses on predicting the exact hazard and product, including their precise relationships in the data.
A score of ~0.425 indicates that the classifier struggles significantly more in this task. This is because predicting both hazards and products correctly is more challenging, and any mismatch in hazards or products (as the compute_score function is stricter when hazards and products are tied) directly impacts the score. The much lower score suggests a significant gap in the classifier’s ability to detect the correct relationships between hazards and products.

  ### Weaknesses:
   - Poor performance on minority classes due to class imbalance.
   - Bias toward majority classes, with low recall for underrepresented categories.

   ### Next steps for improvement if it is verified as the best model:
  - If this model verified as the best one we could try one of the following for better results : 
     1. **Address Class Imbalance:**
   - Use techniques such as oversampling minority classes, undersampling majority classes, or employing weighted loss functions during training.
    - Note : Due to space and time limitations this suggestion is impossible as e.g. SMOTE multiply the already "big" space steeming from vectorizing the text e.g. with the usage of TF-Idf. 

     2. **Focus on Evaluation Metrics:**
   - Utilize metrics like **macro average F1 score** to emphasize balanced performance across all classes.
      - Note: `We will prioritise` this metric.
   - `Pay closer attention to **recall**, especially for minority classes, to improve generalization`.

     3. **Introduce Regularization:**
   - Apply regularization techniques like dropout or L2 regularization to reduce overfitting.
   - Monitor performance on a validation set to detect overfitting during training.
    - Note: Regularization for example in Logistic Regression is represented with the parameter `C` in wich we want to have lo  numbers if we want regularization. 
      - If logistic regression verified as our best model at first glance compared to X-Boost (advanced algorythm) and Random Foresr (simpel algorythm) we will apply hyperparameter tuning emphasizing on this parameter possible values. 
      - We don't adapt it beforehand as our aim is quich to detect our best model firstly. 
- Simillarly we expalin the evaluation reports for the remaining labels. 
- **Note 1**: An F1-score per class equal to 0 means (especially for the product, hazard evaluation reports) the model failed to predict any instances correctly for that class, likely due to no true positives (e.g., it ignored the class or confused it with others), often caused by data imbalance or model limitations.
- **Note 2**: as this satge of benchmark analysis we will not spend time on hyperparamer tuning the model e.g. via intoducing `C` hyperparameter tuned to introduce regularisation as we want a quich overview to find the best model and only fo it we will apply tuning.

# Random Forest  TF-IDF Title

### High-Level Explanation of `train_random_forest_classifiers`

### Objective
- Train Random Forest classifiers for four labels and compute custom metrics to evaluate performance.

### Inputs
- **`dataframe`**: Dataset containing features and target labels.
- **`feature_column`**: The column in the dataframe to be used for feature extraction.

### Outputs
- **`classifiers`**: A dictionary of trained Random Forest classifiers for each label.
- **`vectorizers`**: A dictionary of TF-IDF vectorizers used for feature extraction.
- **`custom_metrics`**: A dictionary of custom evaluation scores for subtasks on test data.

---

### Key Steps

1. **Initialization**:
   - Set a random seed for reproducibility.
   - Prepare dictionaries to store classifiers, vectorizers, and custom metrics.

2. **Label-Specific Training**:
   - Iterate over four labels: 
     - `hazard-category`, 
     - `product-category`, 
     - `hazard`, 
     - `product`.
   - For each label:
     - Perform a stratified train-test split to maintain class distribution.
     - Use `TfidfVectorizer` to extract character-based n-gram features (2-5) with a maximum of 5000 features.
     - Train a Random Forest classifier with 100 decision trees using the extracted TF-IDF features.
     - Evaluate the model using F1 score and generate a classification report.
     - Save the trained classifier, vectorizer, and test data.

3. **Custom Metric Calculation**:
   - Compute task-specific metrics for two subtasks:
     - **Subtask 1**: Evaluate the relationship between `hazard-category` and `product-category`.
     - **Subtask 2**: Evaluate the relationship between `hazard` and `product`.
   - Combine F1 scores for hazards and products to compute the final metric for each subtask.

4. **Logging and Output**:
   - Print F1 scores for each label and custom metric scores for subtasks.
   - Save classification reports to a dedicated directory.
   - Return the trained classifiers, vectorizers, and custom metrics.

---

## Key Techniques
- **TF-IDF Vectorization**: Convert text into feature vectors using character n-grams.
- **Random Forest Classifier**: Train ensemble models with 100 decision trees for robust predictions.
- **Custom Metric Calculation**: Evaluate subtasks by combining F1 scores for hazards and products.

This function enables multi-label text classification and provides task-specific evaluations with a focus on hazards and products.


In [6]:
def train_random_forest_classifiers(dataframe, feature_column):
    """
    Train Random Forest classifiers for four labels and calculate custom metrics on test data.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.

    Returns:
        classifiers: A dictionary containing trained classifiers for each label.
        vectorizers: A dictionary containing TF-IDF vectorizers for each label.
        custom_metrics: A dictionary containing the custom metric score for each pair of labels on test data.
    """
    np.random.seed(42)  # For reproducibility

    classifiers = {}  # Dictionary to store the trained classifiers
    vectorizers = {}  # Dictionary to store the TF-IDF vectorizers
    custom_metrics = {}  # Dictionary to store custom metric scores

    # Dictionaries to store test data for each category
    test_data = {}

    # Train classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Training classifier for label: {label}")

        # Train-test split with stratification based on the current label
        trainset, testset = train_test_split(
            dataframe,
            test_size=0.2,
            random_state=2024,
            stratify=dataframe[label]  # hold proportion of class distribution
        )

        # Extract train and test features
        X_train = trainset[feature_column]
        X_test = testset[feature_column]

        # Target
        y_train = trainset[label]
        y_test = testset[label]

        # Define TfidfVectorizer for the current label
        vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5, max_features=5000)  # limit to 5000
        vectorizers[label] = vectorizer

        # Transform features using the label-specific vectorizer
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Define and train Random Forest classifier
        classifier = RandomForestClassifier(n_estimators=100, random_state=2024, n_jobs=-1)  # Using 100 trees
        classifier.fit(X_train_tfidf, y_train)

        # Store the trained classifier
        classifiers[label] = classifier

        # Store test data
        test_data[label] = {
            'X_test_tfidf': X_test_tfidf,
            'y_test': y_test
        }

        # Predict and evaluate
        predictions = classifier.predict(X_test_tfidf)
        rf_f1 = f1_score(y_test, predictions, average='macro', zero_division=0)
        print(f"F1 Score for {label}: {rf_f1:.3f}")

        # Generate classification report
        report = classification_report(y_test, predictions, zero_division=0)
        print(report)

        # Save the report
        os.makedirs('reports/random_forest', exist_ok=True)
        with open(f'reports/random_forest/rf_classifier_report_{label}_{feature_column}.txt', 'w') as rf_file:
            rf_file.write(f"Classification Report for {label}_{feature_column}:\n")
            rf_file.write(report)
            rf_file.write(f"F1 Score: {rf_f1:.3f}\n")

    # Compute the custom metric for hazards and products using test data only
    custom_metrics['subtask_1'] = compute_score(
        test_data['hazard-category']['y_test'],
        test_data['product-category']['y_test'],
        classifiers['hazard-category'].predict(test_data['hazard-category']['X_test_tfidf']),
        classifiers['product-category'].predict(test_data['product-category']['X_test_tfidf'])
    )

    custom_metrics['subtask_2'] = compute_score(
        test_data['hazard']['y_test'],
        test_data['product']['y_test'],
        classifiers['hazard'].predict(test_data['hazard']['X_test_tfidf']),
        classifiers['product'].predict(test_data['product']['X_test_tfidf'])
    )

    print(f"Custom Metric for Subtask 1 (Test Data): {custom_metrics['subtask_1']:.3f}")
    print(f"Custom Metric for Subtask 2 (Test Data): {custom_metrics['subtask_2']:.3f}")

    return classifiers, vectorizers, custom_metrics


In [7]:
train_random_forest_classifiers(df_augmented, 'title') # call the function "train_random_forest_classifiers"

Training classifier for label: hazard-category


F1 Score for hazard-category: 0.742
                                precision    recall  f1-score   support

                     allergens       0.86      0.94      0.90       863
                    biological       0.84      0.94      0.89       771
                      chemical       0.85      0.85      0.85       329
food additives and flavourings       1.00      0.43      0.60        14
                foreign bodies       0.82      0.75      0.78       253
                         fraud       0.92      0.72      0.81       280
                     migration       1.00      0.93      0.96        29
          organoleptic aspects       1.00      0.26      0.41        39
                  other hazard       0.92      0.55      0.69       103
              packaging defect       1.00      0.36      0.53        39

                      accuracy                           0.86      2720
                     macro avg       0.92      0.67      0.74      2720
                  weighted

({'hazard-category': RandomForestClassifier(n_jobs=-1, random_state=2024),
  'product-category': RandomForestClassifier(n_jobs=-1, random_state=2024),
  'hazard': RandomForestClassifier(n_jobs=-1, random_state=2024),
  'product': RandomForestClassifier(n_jobs=-1, random_state=2024)},
 {'hazard-category': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode'),
  'product-category': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode'),
  'hazard': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode'),
  'product': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode')},
 {'subtask_1': np.float64(0.7602824625659861),
  'subtask_2': np.float64(0.7210352952785184)})

### Quick Evaluation 
- `Custom Metric for Subtask 1 (0.760)`: The model demonstrates strong performance in predicting hazards, with a moderate level of success in predicting associated products, reflecting its reliability for hazard-focused classification tasks.
- `Custom Metric for Subtask 2 (0.721)`: The model performs slightly less effectively in correctly aligning hazards with their associated products, highlighting the increased complexity of this subtask and potential areas for improvement in capturing hazard-product relationships.
- **General Comment** : `Random Forest` has better performance regarding the evaluation custom metric of the competition (for both sub-task 1 and sub-task 2) compared to the Logistic Regression.
    - At this end of bechmark analsyis for "title inout" we will craete a table for summing up all scores in order to be easy the comparison af all models. 
- Note, random Forset outperform baseline models which is fine. 

#  Advanced X-Boost TF-IDF Title 

### High-Level Explanation of `train_xgboost_classifiers`

### Objective
- Train XGBoost classifiers for four labels and compute custom metrics to evaluate performance.

### Inputs
- **`dataframe`**: Dataset containing features and target labels.
- **`feature_column`**: The column in the dataframe to be used for feature extraction.

### Outputs
- **`classifiers`**: A dictionary of trained XGBoost classifiers for each label, including label encoders.
- **`vectorizers`**: A dictionary of TF-IDF vectorizers used for feature extraction.
- **`custom_metrics`**: A dictionary of custom evaluation scores for subtasks on test data.

---

### Key Steps

1. **Initialization**:
   - Set a random seed for reproducibility.
   - Prepare dictionaries to store classifiers, vectorizers, and custom metrics.

2. **Label-Specific Training**:
   - Iterate over four labels:
     - `hazard-category`,
     - `product-category`,
     - `hazard`,
     - `product`.
   - For each label:
     - Perform a stratified train-test split to maintain class distribution.
     - Use `TfidfVectorizer` to extract character-based n-gram features (2-5) with a maximum of 2000 features.
     - Encode target labels into numeric values using `LabelEncoder`.
     - Train an XGBoost classifier with the following parameters:
       - Maximum depth of 6.
       - 50 estimators.
       - Learning rate of 0.2.
     - Evaluate the model using F1 score and generate a classification report.
     - Save the trained classifier, vectorizer, label encoder, and test data.

3. **Custom Metric Calculation**:
   - Compute task-specific metrics for two subtasks:
     - **Subtask 1**: Evaluate the relationship between `hazard-category` and `product-category`.
     - **Subtask 2**: Evaluate the relationship between `hazard` and `product`.
   - Combine F1 scores for hazards and products to compute the final metric for each subtask.

4. **Logging and Output**:
   - Print F1 scores for each label and custom metric scores for subtasks.
   - Save classification reports to a dedicated directory.
   - Return the trained classifiers, vectorizers, and custom metrics.

---

### Key Techniques
- **TF-IDF Vectorization**: Convert text into feature vectors using character n-grams.
- **XGBoost Classifier**: Train scalable and efficient tree-based classifiers with softmax multi-class objectives.
- **Label Encoding**: Map categorical target labels to numeric values for compatibility with XGBoost.
- **Custom Metric Calculation**: Evaluate subtasks by combining F1 scores for hazards and products.

This function enables multi-label text classification and provides task-specific evaluations with a focus on hazards and products using a high-performance gradient boosting model.


In [8]:
def train_xgboost_classifiers(dataframe, feature_column):
    """
    Train XGBoost classifiers for four labels and calculate custom metrics on test data.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.

    Returns:
        classifiers: A dictionary containing trained classifiers for each label.
        vectorizers: A dictionary containing TF-IDF vectorizers for each label.
        custom_metrics: A dictionary containing the custom metric score for each pair of labels on test data.
    """
    np.random.seed(42)  # For reproducibility

    classifiers = {}  # Dictionary to store the trained classifiers
    vectorizers = {}  # Dictionary to store the TF-IDF vectorizers
    custom_metrics = {}  # Dictionary to store custom metric scores

    # Dictionaries to store test data for each category
    test_data = {}

    # Train classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Training classifier for label: {label}")

        # Train-test split with stratification based on the current label
        trainset, testset = train_test_split(
            dataframe,
            test_size=0.2,
            random_state=2024,
            stratify=dataframe[label]  # Maintain class distribution
        )

        # Extract train and test features
        X_train = trainset[feature_column]
        X_test = testset[feature_column]

        # Target
        y_train = trainset[label]
        y_test = testset[label]

        # Encode target labels into numeric values
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)

        # Define TfidfVectorizer for the current label
        vectorizer = TfidfVectorizer(
            strip_accents='unicode', analyzer='char', ngram_range=(2, 5),
            max_df=0.5, min_df=5, max_features=2000
        )
        vectorizers[label] = vectorizer

        # Transform features using the label-specific vectorizer
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Define and train XGBoost classifier
        classifier = XGBClassifier(
            use_label_encoder=False,
            eval_metric='mlogloss',
            objective='multi:softmax',
            max_depth=6,
            n_estimators=50,
            learning_rate=0.2,
            random_state=2024
        )
        classifier.fit(X_train_tfidf, y_train_encoded)

        # Store the trained classifier
        classifiers[label] = {
            'model': classifier,
            'label_encoder': label_encoder
        }

        # Store test data
        test_data[label] = {
            'X_test_tfidf': X_test_tfidf,
            'y_test': y_test_encoded
        }

        # Predict and evaluate
        predictions = classifier.predict(X_test_tfidf)
        xgb_f1 = f1_score(y_test_encoded, predictions, average='macro', zero_division=0)
        print(f"F1 Score for {label}: {xgb_f1:.3f}")

        # Generate classification report
        report = classification_report(y_test_encoded, predictions, zero_division=0, target_names=label_encoder.classes_)
        print(report)

        # Save the report
        os.makedirs('reports/xgboost', exist_ok=True)
        with open(f'reports/xgboost/xgboost_classifier_report_{label}_{feature_column}.txt', 'w') as xgb_file:
            xgb_file.write(f"Classification Report for {label}_{feature_column}:\n")
            xgb_file.write(report)
            xgb_file.write(f"F1 Score: {xgb_f1:.3f}\n")

    # Compute the custom metric for hazards and products using test data only
    custom_metrics['subtask_1'] = compute_score(
        pd.Series(test_data['hazard-category']['y_test']),
        pd.Series(test_data['product-category']['y_test']),
        pd.Series(classifiers['hazard-category']['model'].predict(test_data['hazard-category']['X_test_tfidf'])),
        pd.Series(classifiers['product-category']['model'].predict(test_data['product-category']['X_test_tfidf']))
    )

    custom_metrics['subtask_2'] = compute_score(
        pd.Series(test_data['hazard']['y_test']),
        pd.Series(test_data['product']['y_test']),
        pd.Series(classifiers['hazard']['model'].predict(test_data['hazard']['X_test_tfidf'])),
        pd.Series(classifiers['product']['model'].predict(test_data['product']['X_test_tfidf']))
    )

    print(f"Custom Metric for Subtask 1 (Test Data): {custom_metrics['subtask_1']:.3f}")
    print(f"Custom Metric for Subtask 2 (Test Data): {custom_metrics['subtask_2']:.3f}")

    return classifiers, vectorizers, custom_metrics


- Manually we adapted `n_estimators` and `learning_rate` to run to a logic way. 

In [9]:
classifiers, vectorizers, custom_metrics = train_xgboost_classifiers(df_augmented, 'title')

Training classifier for label: hazard-category


/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:09:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for hazard-category: 0.725
                                precision    recall  f1-score   support

                     allergens       0.84      0.93      0.89       863
                    biological       0.78      0.91      0.84       771
                      chemical       0.81      0.81      0.81       329
food additives and flavourings       1.00      0.43      0.60        14
                foreign bodies       0.87      0.71      0.78       253
                         fraud       0.91      0.69      0.78       280
                     migration       1.00      0.93      0.96        29
          organoleptic aspects       1.00      0.31      0.47        39
                  other hazard       0.90      0.43      0.58       103
              packaging defect       0.94      0.38      0.55        39

                      accuracy                           0.83      2720
                     macro avg       0.90      0.65      0.73      2720
                  weighted

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:10:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for product-category: 0.749
                                                   precision    recall  f1-score   support

                              alcoholic beverages       1.00      0.66      0.79        38
                      cereals and bakery products       0.68      0.63      0.65       252
     cocoa and cocoa preparations, coffee and tea       0.74      0.71      0.72        99
                                    confectionery       0.87      0.53      0.66        91
dietetic foods, food supplements, fortified foods       0.91      0.79      0.85        95
                                    fats and oils       0.88      0.72      0.79        32
                                   feed materials       0.91      0.91      0.91        11
                   food additives and flavourings       1.00      0.60      0.75        10
                           food contact materials       0.94      0.89      0.91        35
                            fruits and vegetables   

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:12:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for hazard: 0.668
                                                   precision    recall  f1-score   support

                                        Aflatoxin       1.00      0.60      0.75         5
                                   abnormal smell       1.00      1.00      1.00         7
                                  alcohol content       0.91      1.00      0.95        10
                                        alkaloids       1.00      0.89      0.94         9
                                        allergens       1.00      0.29      0.44         7
                                           almond       0.82      0.58      0.68        31
             altered organoleptic characteristics       0.82      0.90      0.86        10
                                        amygdalin       0.88      0.78      0.82         9
                           antibiotics, vet drugs       1.00      0.33      0.50         6
                                    bacillus spp.       0.80  

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:18:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for product: 0.632
                                                                        precision    recall  f1-score   support

                                                Catfishes (freshwater)       1.00      1.00      1.00         3
                                                       Dried pork meat       1.00      1.00      1.00         1
                                                 Fishes not identified       0.38      0.33      0.35         9
                                                    Groupers (generic)       1.00      1.00      1.00         1
                                              Not classified pork meat       1.00      0.75      0.86         4
                                            Pangas catfishes (generic)       1.00      0.50      0.67         2
                                   Precooked cooked pork meat products       0.75      0.50      0.60         6
                                    Torpedo-shaped catfishes (generic)     

- We can observe via focusing on custom competition score taht X-Boost has lower performance than  random Forest for the input "title".
- In the following table we have summ up all scores. 


### Sum Up for benchmark Analysis for "Title" Input : 

|     | `Sub Task 1`     | `Sub Task 2`    |
|--------------|--------------|--------------|
| `Random Baseline`| 0.057 | 0.003 |
| `Majority Baseline`| 0.031 | 0.001 |
| `LogisticRegression Title` | 0.690 | 0.425 |
| `Random Forest Title` | `0.760` | `0.721` |
| `X-Boost Title`| 0.741 | 0.647 |

- The columns of the Table above have the custom scores of the competition for teh sub task 1 and sub task 2.
- **Notes / Observations / Commnet** : 
    - Both traditional (Logistic Regression and Ranfdom Forest) and advanced mdel (X-Boost) models outperform the baseline models (Random and Majority, which is fine).
    - The best potential of "best" score in the sub-task 1 and sub-task 2 seem to be presented with `Random Forest` classifier. Then follows X-Boost having good potential, too. Logistic regression is out of competition as especially for the sub task 2 the score is extremely low (0.425) 
- We will repeat the benchmark anaysis for the text as imput. If random Forest with "title" is improved as the best algorythm, we could try further tuning with a systematic way this time using e.h. greed search in combination with cross validation to have robostness in the evaluation metrics.
- In addition, in case with have time we could group for the hazards and hazards-category labels the classes having low f-score in a category Other (this is the value of the evaluation report in teh scope of the competition).

### Part B. Benchmark Analysis Text

- **Important** note, because `text` has higher length than "title" we treid to adapt manually (for leveraging time) the parameter of tf-idf `max_feature`, but the score did not improved. 
- So, we concluded to simply re-call the aforementioned functions changing only the feature from "title" to "text" and comapre the custom scores of the competition, too.
- Note: Having run the three models at the end of the process we will generate a similar table like in title input ans comapre the models at the end. 
- Firslty, we will call the function `train_log_regression_classifiers` via pssign "text" as input. 


In [14]:
classifiers, vectorizers, custom_metrics = train_log_regression_classifiers(df_augmented, 'text')

Training classifier for label: hazard-category


/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for hazard-category: 0.730
                                precision    recall  f1-score   support

                     allergens       0.93      0.98      0.95       863
                    biological       0.93      0.96      0.94       771
                      chemical       0.81      0.93      0.86       329
food additives and flavourings       1.00      0.29      0.44        14
                foreign bodies       0.83      0.86      0.84       253
                         fraud       0.77      0.69      0.73       280
                     migration       1.00      1.00      1.00        29
          organoleptic aspects       1.00      0.18      0.30        39
                  other hazard       0.86      0.55      0.67       103
              packaging defect       0.94      0.38      0.55        39

                      accuracy                           0.89      2720
                     macro avg       0.91      0.68      0.73      2720
                  weighted

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for product-category: 0.659
                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.89      0.66      0.76        38
                      cereals and bakery products       0.51      0.63      0.56       252
     cocoa and cocoa preparations, coffee and tea       0.73      0.46      0.57        99
                                    confectionery       0.82      0.41      0.54        91
dietetic foods, food supplements, fortified foods       0.77      0.71      0.74        95
                                    fats and oils       0.86      0.56      0.68        32
                                   feed materials       1.00      0.73      0.84        11
                   food additives and flavourings       1.00      0.70      0.82        10
                           food contact materials       1.00      0.97      0.99        35
                            fruits and vegetables   

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for hazard: 0.500
                                                   precision    recall  f1-score   support

                                        Aflatoxin       0.00      0.00      0.00         5
                                   abnormal smell       1.00      0.71      0.83         7
                                  alcohol content       0.83      1.00      0.91        10
                                        alkaloids       1.00      0.67      0.80         9
                                        allergens       0.00      0.00      0.00         7
                                           almond       0.73      0.52      0.60        31
             altered organoleptic characteristics       1.00      0.80      0.89        10
                                        amygdalin       0.75      1.00      0.86         9
                           antibiotics, vet drugs       0.00      0.00      0.00         6
                                    bacillus spp.       1.00  

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1 Score for product: 0.337
                                                                        precision    recall  f1-score   support

                                                Catfishes (freshwater)       0.00      0.00      0.00         3
                                                       Dried pork meat       0.00      0.00      0.00         1
                                                 Fishes not identified       0.14      0.44      0.22         9
                                                    Groupers (generic)       0.00      0.00      0.00         1
                                              Not classified pork meat       0.00      0.00      0.00         4
                                            Pangas catfishes (generic)       0.00      0.00      0.00         2
                                   Precooked cooked pork meat products       1.00      0.33      0.50         6
                                    Torpedo-shaped catfishes (generic)     

- Now, we will call the fucntion `train_random_forest_classifiers` with input `text` 

In [11]:
train_random_forest_classifiers(df_augmented, 'text')

Training classifier for label: hazard-category
F1 Score for hazard-category: 0.770
                                precision    recall  f1-score   support

                     allergens       0.94      1.00      0.97       863
                    biological       0.96      0.96      0.96       771
                      chemical       0.83      0.92      0.87       329
food additives and flavourings       1.00      0.29      0.44        14
                foreign bodies       0.82      0.91      0.87       253
                         fraud       0.84      0.76      0.80       280
                     migration       1.00      1.00      1.00        29
          organoleptic aspects       1.00      0.23      0.38        39
                  other hazard       0.94      0.66      0.78       103
              packaging defect       0.95      0.49      0.64        39

                      accuracy                           0.91      2720
                     macro avg       0.93      0.72

({'hazard-category': RandomForestClassifier(n_jobs=-1, random_state=2024),
  'product-category': RandomForestClassifier(n_jobs=-1, random_state=2024),
  'hazard': RandomForestClassifier(n_jobs=-1, random_state=2024),
  'product': RandomForestClassifier(n_jobs=-1, random_state=2024)},
 {'hazard-category': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode'),
  'product-category': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode'),
  'hazard': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode'),
  'product': TfidfVectorizer(analyzer='char', max_df=0.5, max_features=5000, min_df=5,
                  ngram_range=(2, 5), strip_accents='unicode')},
 {'subtask_1': np.float64(0.7841164075998285),
  'subtask_2': np.float64(0.7580341615326311)})

- we will call the function `train_xgboost_classifiers` with input `text`

In [10]:
classifiers, vectorizers, custom_metrics = train_xgboost_classifiers(df_augmented, 'text')

Training classifier for label: hazard-category


/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:36:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for hazard-category: 0.840
                                precision    recall  f1-score   support

                     allergens       0.95      0.99      0.97       863
                    biological       0.96      0.97      0.97       771
                      chemical       0.88      0.96      0.91       329
food additives and flavourings       1.00      0.57      0.73        14
                foreign bodies       0.87      0.94      0.90       253
                         fraud       0.91      0.78      0.84       280
                     migration       1.00      0.97      0.98        29
          organoleptic aspects       1.00      0.46      0.63        39
                  other hazard       0.90      0.74      0.81       103
              packaging defect       0.91      0.51      0.66        39

                      accuracy                           0.93      2720
                     macro avg       0.94      0.79      0.84      2720
                  weighted

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:39:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for product-category: 0.788
                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.96      0.66      0.78        38
                      cereals and bakery products       0.60      0.74      0.66       252
     cocoa and cocoa preparations, coffee and tea       0.81      0.70      0.75        99
                                    confectionery       0.89      0.46      0.61        91
dietetic foods, food supplements, fortified foods       0.91      0.67      0.78        95
                                    fats and oils       1.00      0.66      0.79        32
                                   feed materials       1.00      0.82      0.90        11
                   food additives and flavourings       1.00      0.80      0.89        10
                           food contact materials       1.00      0.91      0.96        35
                            fruits and vegetables   

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:43:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for hazard: 0.823
                                                   precision    recall  f1-score   support

                                        Aflatoxin       1.00      0.80      0.89         5
                                   abnormal smell       1.00      1.00      1.00         7
                                  alcohol content       0.91      1.00      0.95        10
                                        alkaloids       0.90      1.00      0.95         9
                                        allergens       1.00      0.57      0.73         7
                                           almond       0.84      0.84      0.84        31
             altered organoleptic characteristics       1.00      1.00      1.00        10
                                        amygdalin       1.00      1.00      1.00         9
                           antibiotics, vet drugs       1.00      0.67      0.80         6
                                    bacillus spp.       1.00  

/Users/evangelia/Desktop/neo-ergasia-2/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:53:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score for product: 0.697
                                                                        precision    recall  f1-score   support

                                                Catfishes (freshwater)       0.33      0.33      0.33         3
                                                       Dried pork meat       1.00      1.00      1.00         1
                                                 Fishes not identified       0.00      0.00      0.00         9
                                                    Groupers (generic)       1.00      1.00      1.00         1
                                              Not classified pork meat       0.50      0.75      0.60         4
                                            Pangas catfishes (generic)       1.00      0.50      0.67         2
                                   Precooked cooked pork meat products       1.00      0.83      0.91         6
                                    Torpedo-shaped catfishes (generic)     

- Observing mostly the evaluation scores of the competitions for sun task 1 and sub task 2, we concluded that X-Boost has more potential than random forest and logistic multinomial regression.
- So, we will try to hyperparameter tuned it.

### Sum Up for benchmark Analysis for "Title" Input :

- In the teable below we have sum up the results of the models bot for 'title" and "text" and we have make "yellow" teh best scores for "tile" as input and for "text" as input 

|     | `Sub Task 1`     | `Sub Task 2`    |
|--------------|--------------|--------------|
| `Random Baseline`| 0.057 | 0.003 |
| `Majority Baseline`| 0.031 | 0.001 |
| `LogisticRegression Title` | 0.690 | 0.425 |
| `Random Forest Title` | `0.760` | `0.721` |
| `X-Boost Title`| 0.741 | 0.647 |
| `LogisticRegression Text` | 0.695 | 0.427 |
| `Random Forest Tetx` | 0.784 | 0.758 |
| `X-Boost Text`| `0.814` | `0.759` |

- For the `ttile` the best algorythm is `Random Forest`
- Whereas, for "text" as input the best algorythm is `x-boost`
- So, we will buld on x-boost with input as text and we will try to hyper tuning it more for better results. 
- Note :All algorythms outperform the baselines. 
- In depth analysis for the evalaution report only for the best algorythm with the best input (taht is  `X-Boost with input text`): 
    - because the sub task 1 and sub task 2 are affected more than the labels related to `hazard` (that is labels: `hazard` and `hazard category` ) we could leverage (if we have time) to group the classes with low f1 score to a category other and rerun the process. E.g. for the `hazard category` we could try to group  organoleptic aspects and packaging defect as they have the lowest f1 score scores in comparison to the remaining classes (about 0.65) and the same strategy t ofollow for the `hazard` label too. 
- In addition regardign the input, we can observe that al classifiers has better scores with input the "text". 
### Custom "best" parameters with manually tuning for X-Boost and TF-idf to build on 

- `X-boost` has the following `parameters` : 

```bash
   classifier = XGBClassifier(
            use_label_encoder=False,
            eval_metric='mlogloss',
            objective='multi:softmax',
            max_depth=6,
            n_estimators=50,
            learning_rate=0.2,
            random_state=2024
        )
```
- And `TF-idf` the follwoing `parameters` :

```bash
TfidfVectorizer(
            strip_accents='unicode', analyzer='char', ngram_range=(2, 5),
            max_df=0.5, min_df=5, max_features=2000
        )
```

- **`Important Note`**:  even in the scope of benchmark analysis the "professioanl" approach to adapt the parameters of both X-Boost and TF-idf would be using Grid Search or MLOps like Optuna so as systematically to find the best combiantion.
    - Because manual adapatation may causee the suspission of the work. 
    - However, duw to time and memory limitations we cannot hyperparrameter tuning for all models. 
    - We tried for the given time to adapt them manually bearing in mind the nature of the problem (many classes) and check mostly teh performance of competition scores for both sub-task 1 and sub-task 2.
    - In addition, the advantage of systematic hyperparameter tunign is that we can apply cross validation (which is time cosuming) in order to have better overiview about the robostness of the performance metrcis. 

## Part 3 Improve best benchmark model - X-Boost (input text)

### Train XGBoost Classifiers with TF-IDF and Hyperparameter Tuning

### **Overview**
- This function trains and evaluates XGBoost classifiers for multi-label text classification using TF-IDF features and hyperparameter tuning with `GridSearchCV`.
---

### **Function Inputs**
1. **`dataframe`**: A pandas DataFrame containing the dataset.
2. **`feature_column`**: The name of the column containing the text features.

---

### **Function Outputs**
1. **`classifiers`**: 
   - A dictionary of trained XGBoost classifiers and their corresponding label encoders.
2. **`vectorizers`**:
   - A dictionary of TF-IDF vectorizers for transforming text data.
3. **`custom_metrics`**:
   - A dictionary of custom metrics for specific subtasks.

---

### **Workflow**

### **1. Preprocessing**
- Splits the dataset into train and test sets with stratification.
- Encodes categorical labels into numeric values (`LabelEncoder`).
- Converts text data into TF-IDF features using `TfidfVectorizer`:
  - Character n-grams (`2-5`).
  - Maximum features: `2000`.
  - Filters by document frequency (`min_df=5`, `max_df=0.5`).

### **2. Model Training**
- Trains separate XGBoost classifiers for the following labels:
  - `hazard-category`
  - `product-category`
  - `hazard`
  - `product`
- Uses `GridSearchCV` for hyperparameter tuning:
  - Parameters: `max_depth`, `n_estimators`, `learning_rate`.
  - Optimizes `f1_macro` with 3-fold cross-validation.

### **3. Evaluation**
- Computes `f1_score` for macro averaging.
- Generates and saves `classification_report` for each label.

### **4. Custom Metrics**
- Computes task-specific metrics using test data for:
  - **Subtask 1**: `hazard-category` and `product-category`.
  - **Subtask 2**: `hazard` and `product`.

---

### **Features**
- **Multi-label Classification**:
  - Trains and evaluates separate models for each label.
- **TF-IDF Feature Extraction**:
  - Efficiently transforms text data.
- **Hyperparameter Tuning**:
  - Ensures optimal model performance using `GridSearchCV`.
- **Report Generation**:
  - Saves evaluation reports for auditability.

---

### **Limitations**
- **Performance**:
  - Grid search can be computationally expensive for large datasets.




In [7]:
from sklearn.model_selection import GridSearchCV
def train_xgboost_classifiers_with_tuning(dataframe, feature_column):
    """
    Train XGBoost classifiers with hyperparameter tuning for XGBoost
    for four labels and calculate custom metrics on test data.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.

    Returns:
        classifiers: A dictionary containing trained classifiers for each label.
        vectorizers: A dictionary containing TF-IDF vectorizers for each label.
        custom_metrics: A dictionary containing the custom metric score for each pair of labels on test data.
    """
    np.random.seed(42)  # For reproducibility

    classifiers = {}  # Dictionary to store the trained classifiers
    vectorizers = {}  # Dictionary to store the TF-IDF vectorizers
    custom_metrics = {}  # Dictionary to store custom metric scores

    # Dictionaries to store test data for each category
    test_data = {}

    # Define the hyperparameter grid for XGBoost
    param_grid_xgb = {
        'max_depth': [7, 8],#before 6 compare
        'n_estimators': [50],
        'learning_rate': [0.2],

    }

    # Train classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Training classifier for label: {label}")

        # Train-test split with stratification based on the current label

        trainset, testset = train_test_split(
            dataframe,
            test_size=0.2,
            random_state=2024,
            stratify=dataframe[label]  # Maintain class distribution
        )

        # Extract train and test features
        X_train = trainset[feature_column]
        X_test = testset[feature_column]

        # Target
        y_train = trainset[label]
        y_test = testset[label]

        # Encode target labels into numeric values
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)

        # Define TfidfVectorizer for the current label
        vectorizer = TfidfVectorizer(
            strip_accents='unicode', analyzer='char', ngram_range=(2, 5),
            max_df=0.5, min_df=5, max_features=2000
        )
        vectorizers[label] = vectorizer

        # Transform features using the label-specific vectorizer
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Define the XGBoost classifier
        base_classifier = XGBClassifier(
            # use_label_encoder=False,
            eval_metric='mlogloss',
            objective='multi:softmax',
            random_state=2024
        )

        grid_search = GridSearchCV(
            estimator=base_classifier,
            param_grid=param_grid_xgb,
            scoring='f1_macro',
            cv=3, # cross  validation 3 more robostness results 
            verbose=2,
            n_jobs=-1


        )
        grid_search.fit(X_train_tfidf, y_train_encoded)

        # Get the best model
        best_classifier = grid_search.best_estimator_

        # Store the trained classifier
        classifiers[label] = {
            'model': best_classifier,
            'label_encoder': label_encoder
        }

        # Store test data
        test_data[label] = {
            'X_test_tfidf': X_test_tfidf,
            'y_test': y_test_encoded
        }

        # Predict and evaluate
        predictions = best_classifier.predict(X_test_tfidf)
        xgb_f1 = f1_score(y_test_encoded, predictions, average='macro', zero_division=0)
        print(f"F1 Score for {label}: {xgb_f1:.3f}")

        # Generate classification report
        report = classification_report(y_test_encoded, predictions, zero_division=0, target_names=label_encoder.classes_)
        print(report)

        # Save the report
        os.makedirs('reports/xgboost', exist_ok=True)
        with open(f'reports/xgboost/xgboost_classifier_report_{label}_{feature_column}_tuning.txt', 'w') as xgb_file:
            xgb_file.write(f"Classification Report for {label}_{feature_column}_tuning:")
            xgb_file.write(report)
            xgb_file.write(f"F1 Score: {xgb_f1:.3f}\n")

    # Compute the custom metric for hazards and products using test data only
    custom_metrics['subtask_1'] = compute_score(
        pd.Series(test_data['hazard-category']['y_test']),
        pd.Series(test_data['product-category']['y_test']),
        pd.Series(classifiers['hazard-category']['model'].predict(test_data['hazard-category']['X_test_tfidf'])),
        pd.Series(classifiers['product-category']['model'].predict(test_data['product-category']['X_test_tfidf']))
    )

    custom_metrics['subtask_2'] = compute_score(
        pd.Series(test_data['hazard']['y_test']),
        pd.Series(test_data['product']['y_test']),
        pd.Series(classifiers['hazard']['model'].predict(test_data['hazard']['X_test_tfidf'])),
        pd.Series(classifiers['product']['model'].predict(test_data['product']['X_test_tfidf']))
    )

    print(f"Custom Metric for Subtask 1 (Test Data): {custom_metrics['subtask_1']:.3f}")
    print(f"Custom Metric for Subtask 2 (Test Data): {custom_metrics['subtask_2']:.3f}")

    return classifiers, vectorizers, custom_metrics


In [8]:
classifiers, vectorizers, custom_metrics = train_xgboost_classifiers_with_tuning(df_augmented, 'text')

Training classifier for label: hazard-category
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END ....learning_rate=0.2, max_depth=7, n_estimators=50; total time= 5.9min
[CV] END ....learning_rate=0.2, max_depth=7, n_estimators=50; total time= 5.9min
[CV] END ....learning_rate=0.2, max_depth=7, n_estimators=50; total time= 5.9min
[CV] END ....learning_rate=0.2, max_depth=8, n_estimators=50; total time= 6.4min
[CV] END ....learning_rate=0.2, max_depth=8, n_estimators=50; total time= 6.4min
[CV] END ....learning_rate=0.2, max_depth=8, n_estimators=50; total time= 6.4min
F1 Score for hazard-category: 0.840
                                precision    recall  f1-score   support

                     allergens       0.95      0.99      0.97       863
                    biological       0.97      0.98      0.97       771
                      chemical       0.89      0.95      0.92       329
food additives and flavourings       0.80      0.57      0.67        14
           

- Note the output above reflects the process of heperparameter tuning in combination with applied cross validation during tuning 
    ```bash
    [CV] END ....learning_rate=0.2, max_depth=7, n_estimators=50; total time= 5.9min
    [CV] END ....learning_rate=0.2, max_depth=7, n_estimators=50; total time= 5.9min
    [CV] END ....learning_rate=0.2, max_depth=7, n_estimators=50; total time= 5.9min
    [CV] END ....learning_rate=0.2, max_depth=8, n_estimators=50; total time= 6.4min
    [CV] END ....learning_rate=0.2, max_depth=8, n_estimators=50; total time= 6.4min
    [CV] END ....learning_rate=0.2, max_depth=8, n_estimators=50; total time= 6.4min
- For example, during the hyperparameter tuning of the first "individual" label that is `hazard-category` we can observe that the `max_depth` hyperparameters systematically adapted acrooss cross validations stages (which is more robosmenees adapattion because it is checked in diffrent spitted sub datasets).
    - From the intiial depth (6) we systematically tried using both cross validation with greed search to investigate a better hyperparameter for max depth and we conluded with `max_depth=8`
    - For all individual models the max_depth adapted to 8.
- Compare results of X-boost text with **tuned** X-Boost text: 


|     | `Sub Task 1`     | `Sub Task 2`    |
|--------------|--------------|--------------|
| X-Boost (`max_depth=6`) Text| 0.814 | 0.759 |
| tuned X-Boost (`max_depth=8`) Text | 0.815 | `0.762`|

- We can observe a sligh improvement especially for the score of sub-task 2.
- We will make a first prediction with the unlabelled data and if we have time we will try more systematic tuning  for both x-boost parameters and tf-df (here we could try to increase it more and then apply PCA to reduce teh diamensionality).

## Prediction of Unlabeled Data with our best Classifier X-Boost (max_depth=8 for all y  - labels).

- Firslty, we will predict `hazard` and `product` for the sub task 2.
- Note, wheter or not apply basic nlp preprocess to the unlabelled data the scores are not diffrent significantly, a bot betetr score we recieve if we pass the unlabelled data in baseic nlp preprocess.
- Firstly, we will Access specific classifiers of X-Boost - Tuned as it is common for both the prediction fo sub task 1 and sub task 2.

In [64]:
hazard_classifier = classifiers['hazard'] # access individual classifier for "hazard"
product_classifier = classifiers['product'] # access indvidua lcassifier for "product"
hazard_category_classifier = classifiers['hazard-category'] # access individual classfiier for "hazard-category"
product_category_classifier = classifiers['product-category'] # access individual classifier for "product-category" 

- Note General: The nested dictionary format we used to store the resulats of our classifiers and tf-idf permit us to delve into them and observe their context.
- Regardign the tf-idf`, the following cell shows us the parameters of tf-idf used being : 
    - analyzer=`char`, max_df=`0.5`, `max_features=2000`, `min_def=5`, `nram_range=(2,5)`, `strip_accents='unicode`.

In [65]:
vectorizers['hazard']

TfidfVectorizer(analyzer='char', max_df=0.5, max_features=2000, min_df=5,
                ngram_range=(2, 5), strip_accents='unicode')

- Note, on teh second thought for the prediction it would be better if we had implemented tf-idf and model as pipeline, in order not to vectorize seperately the unlabeleed data. 

### Predict `Hazard` X-Boost (hyperparameter tuned with max_depth=8) Input: text

- Firslty, we will vectorize the unlabelled data.

In [66]:
vectorizer = vectorizers['hazard']#pass TF-idf of training TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5,max_features=2000)
vectorizer.fit_transform(df_augmented['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val=vectorizer.transform(testset_competition['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val # vectorized X input of unlabeled data the column "text"

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 251090 stored elements and shape (565, 2000)>

- Then we will save the individual model foe the hazard label and the label_encoder (as x-boost have to encoded the y classes from string format to numebrs in order to predict them).

In [67]:
model_hazard_classifier = hazard_classifier['model'] # save trained model X-Boost 
label_encoder_hazard_classifier = hazard_classifier['label_encoder'] # Save trained model label_encoder 

- We can delve into the  context of x-boost , in order to see the valeus of the hyperparameters we defienedm the remaining ones will have "none".

In [68]:
model_hazard_classifier

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

- Note : If we had the needed computational power and not limited time for the deadliene of this assignemnt, we had to tune for more hyp-parameters to find the best combinations maybe with the usage of an MLOps like `Optuna` instaead of `greed search` the process of hyperparameter tuning would be better. 
    - e.g. `early stopping` is a fine hyperparamet for avoid model from overfitting. Because we did not defined it is has the value None. We focused on `max_depth` only to show the logic of hyperparameter tuning.
- In addition due to cross-val (K fold 3 in our case) the process speed down more.
- We will now print the contaxt of label encoder of x-boost 


In [69]:
label_encoder_hazard_classifier

LabelEncoder()

- it is teh default one.
- if we predict with our model the unlabeled data we can see taht it returns numebrs instead of string this is due to x-boost label encoder. 

In [70]:
model_hazard_classifier.predict(X_val)

array([ 55,  94,  90,  99, 106, 101,  97,  59,  57, 101,  58,  36,  36,
        78, 101, 104,  31,  36,  39,  98,  22, 101,  39,  39,  40,  39,
        30,  73, 101,  75,  22, 101,  59,  94,  25,  25,  39,  58,  39,
        39,  32,  75,  73, 110,  80,  12, 127, 101,  39,  32, 127, 127,
       104,  97, 101,  73,  73,  90,  32, 101,  39,  32, 101,  31,  90,
        80,  90,  27,  57,  73,  74,  90,  22,  59,  27, 104,  39,  14,
        35,  33,  73,  78,  85,  97,  73, 101,  90,  34, 110,  98,  27,
        90,  97, 101,  39,  94, 101,  27,  74,  39, 110,  73, 101,  97,
        66,  90,  74,  73,  73, 101,  21,  78,  47,  47,  90, 102,  39,
        90,  51,  51,  27,  83,  31, 104,  74,  59,  47,  90,  47,  39,
        13,  27,  57,  90,  90,  39,  30,  22,  51,  99,  73,  73,   5,
        85,  25,  64,  47,  14,   5,  36, 124,  73,  47,  61,  47,  73,
        73,  57,  57,  57,  57,  47,  99,  27,  59,  83,  73,  59, 104,
        83,  73,  85,  27,  78,  58,  34,  90,  32,  23,  97,  4

In [71]:
model_hazard_classifier.predict(X_val).shape[0]
predictions_hazard= model_hazard_classifier.predict(X_val)

- with the following command we convert number format to string format (`inverse_transform`).

In [72]:
predictions_named_hazard = label_encoder_hazard_classifier.inverse_transform(predictions_hazard)  # Get original class names        predictions_dict[label] = predictions_named  # Store predictions with names

- In the cell below we can see the prediction in the format of string. 

In [73]:
predictions_named_hazard

array(['listeria monocytogenes', 'processing', 'plastic fragment',
       'sesame seeds and products thereof', 'sulfamethazine unauthorised',
       'specified risk material (srm)', 'rubber fragments',
       'milk and products thereof', 'metal fragment',
       'specified risk material (srm)', 'microbiological contamination',
       'escherichia coli', 'escherichia coli',
       'other not classified chemical hazards',
       'specified risk material (srm)', 'staphylococcus',
       'damaged packaging', 'escherichia coli', 'foreign bodies',
       'salmonella', 'clostridium botulinum',
       'specified risk material (srm)', 'foreign bodies',
       'foreign bodies', 'glass fragment', 'foreign bodies',
       'dairy products', 'other', 'specified risk material (srm)',
       'other not classified', 'clostridium botulinum',
       'specified risk material (srm)', 'milk and products thereof',
       'processing', 'compositional deviation', 'compositional deviation',
       'foreign bodi

In [74]:
predictions_named_hazard.shape[0]

565

- until here we have compmeted the prediction of `hazard` label the context of which has been saved to the variable `predictions_named_hazard`.
- we will move on with the labe; `product` in order to complete the prediction of `sub task 2`

### Sub Task 2: Predict Product X-Boost (hyperparameter tuned with max_depth=8) Input: text

- In short, we will repeat simillar process with the aforementioned one so will will not re-analase it.
- We will begin with vectorization, predicttion, label encoder. 

In [94]:
product_classifier = classifiers['product']
vectorizer = vectorizers['product']#pass TF-idf of training TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5,max_features=2000)
vectorizer.fit_transform(df_augmented['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val=vectorizer.transform(testset_competition['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val # vectorized X input of unlabeled data the column "text"

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 251090 stored elements and shape (565, 2000)>

In [95]:
model_product_classifier = product_classifier['model'] # save trained model X-Boost 
label_encoder_product_classifier = product_classifier['label_encoder'] # Save trained model label_encoder 
predictions_product= model_product_classifier.predict(X_val)
predictions_product

array([ 902,  825,  902,  870,  346,   58,  292,  742,  150,  641,  118,
        150,  117,  260,  150,  150,  150,  280,  665,  990,  150,   58,
        763,   73,  665,  665,   58,   58,  117,  150,   58,   28,  150,
        588,  260,  260,  150,  260,  665,   58,  150,  150,  150,  270,
        334,  932,   58,  690,  150,  902,  902,  260,  588,  260,  150,
         58,   58,  150,  150,  117,  588,   58,  839,  150,  782,  520,
        530,  588,  150,   49,   58,  150,  300,  588,  588,  150,  150,
        150,  127,  212,  150, 1014,  292,  150,  802,  117,  150,  950,
        588,  990,  588,  968,  713,  904,  150,  150,   38,  588,  870,
        260,  990,  814,  665,  865,  284,  588,  280,  150,  334,  260,
         28,  150,  832,  711,  150,  284,  588,  665,  782,  842,  150,
        117,  588,  520,  690,  117,  899,  260,  304,  298,  604,  990,
        711,  822,  260,  260,  530,  814,  899,  662,  578,  260,  588,
        588, 1020,  588,  150,  420,  117,  304,  1

In [96]:
product_predictions_named = label_encoder_product_classifier.inverse_transform(predictions_product)  # Get original class names        predictions_dict[label] = predictions_named  # Store predictions with names
product_predictions_named

array(['sprouts', 'sausage', 'sprouts', 'soup', 'eggs', 'beef', 'custard',
       'precooked cooked beef meat products', 'cheese', 'nuts',
       'camembert', 'cheese', 'cakes', 'cookies', 'cheese', 'cheese',
       'cheese', 'crushed red pepper',
       'other not classified meat products', 'vegetable samosas',
       'cheese', 'beef', 'pumpkin seeds', 'biscuits',
       'other not classified meat products',
       'other not classified meat products', 'beef', 'beef', 'cakes',
       'cheese', 'beef', 'apple juice', 'cheese',
       'mechanically separated chicken meat', 'cookies', 'cookies',
       'cheese', 'cookies', 'other not classified meat products', 'beef',
       'cheese', 'cheese', 'cheese', 'crackers', 'dried strawberries',
       'thermal processed beef meat', 'beef', 'peanuts', 'cheese',
       'sprouts', 'sprouts', 'cookies',
       'mechanically separated chicken meat', 'cookies', 'cheese', 'beef',
       'beef', 'cheese', 'cheese', 'cakes',
       'mechanically separat

- Having both predict he `hazard` amnd `product` we have completed teh prediction for the subb task **2**.
- So, we will save the predictions to a csv in order to sub,mit it to teh compettiion paltform.  

### Save CSV product & hazartd (ST2)

- We will save the results to the folder / file :  ` "data_augmented_submission/st2_initial"`
    - manually we rename it to be in the acceptable competition format.

In [99]:
# Create folder structure if it doesn't exist
base_dir = "data_augmented_submission/st2_initial"
os.makedirs(base_dir, exist_ok=True)
# Hazard product data
data_st1 = {
    "hazard": predictions_named_hazard,
    "product": product_predictions_named
}
df_st1 = pd.DataFrame(data_st1)
# Save to a CSV
filecsv_file_path = os.path.join(base_dir, "submission.csv")
df_st1.to_csv(filecsv_file_path, index=False) # save to csv 

### Subtask 1 Predict Hazard and Product Categories (Sub Task 1)

- Now we will make the predstions for the unmabeled data for the input "text" for the sub task 1 related to `hzard=category` and `product-category`.
- we will repeat the same proess (vectorization to the unlabeled, prediction, label encoder).

In [101]:
hazard_category_classifier = classifiers['hazard-category'] # define individual classifier for hazard-category
product_category_classifier = classifiers['product-category'] # define individual classfifier for product-category
vectorizer = vectorizers['hazard-category']#pass TF-idf of training TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5,max_features=2000)
vectorizer.fit_transform(df_augmented['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val=vectorizer.transform(testset_competition['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val # vectorized X input of unlabeled data the column "text"

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 251090 stored elements and shape (565, 2000)>

In [102]:
model_hazard_category_classifier = hazard_category_classifier['model'] # save trained model X-Boost 
label_encoder_hazard_category_classifier = hazard_category_classifier['label_encoder'] # Save trained model label_encoder 
model_hazard_category_classifier

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

- Here we can see again all hyperparameters that X-Boost accepts, only `max_depth` and `learning rate`, `n_estimators` has values as for greed search and in our initial intalitaion fof X-Boost model these defines / searched. 

In [103]:
label_encoder_hazard_category_classifier

LabelEncoder()

In [104]:
model_hazard_category_classifier.predict(X_val) # due to label encoder the strings has been converted to numbers 

array([2, 2, 2, 1, 2, 4, 1, 4, 4, 8, 4, 4, 1, 4, 4, 1, 0, 4, 4, 1, 1, 1,
       4, 5, 8, 4, 2, 4, 0, 0, 4, 8, 0, 1, 1, 1, 0, 4, 4, 0, 4, 4, 4, 1,
       4, 0, 4, 0, 2, 4, 4, 4, 2, 4, 1, 4, 8, 4, 8, 2, 4, 4, 4, 1, 1, 1,
       4, 1, 1, 4, 0, 0, 5, 1, 1, 1, 4, 2, 2, 4, 1, 2, 0, 1, 1, 1, 1, 0,
       1, 1, 4, 5, 0, 4, 0, 2, 1, 1, 1, 0, 1, 2, 1, 1, 0, 1, 8, 1, 0, 0,
       1, 4, 0, 0, 1, 0, 0, 4, 1, 1, 0, 1, 1, 1, 1, 0, 0, 2, 0, 4, 1, 1,
       1, 0, 0, 0, 0, 2, 0, 0, 4, 0, 1, 0, 0, 1, 0, 1, 1, 1, 4, 4, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 4, 0, 1, 1, 1, 0,
       0, 0, 2, 1, 4, 1, 1, 4, 0, 0, 0, 0, 0, 1, 4, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 4, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 8, 2, 9, 1, 0, 1, 1, 1, 0, 1, 8, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 4, 2, 0, 8, 8, 4, 1, 7, 2, 0, 2, 0, 1, 0, 1, 4, 8,
       1, 0, 0, 1, 1, 8, 0, 0, 0, 4, 5, 2, 8, 0, 5,

In [105]:
predictions_hazard_category= model_hazard_category_classifier.predict(X_val)

In [106]:
predictions_named_hazard = label_encoder_hazard_classifier.inverse_transform(predictions_hazard_category)  # Get original class names redictions_dict[label] = predictions_named  # Store predictions with names
predictions_named_hazard

array(['alcohol content', 'alcohol content', 'alcohol content',
       'abnormal smell', 'alcohol content', 'allergens', 'abnormal smell',
       'allergens', 'allergens', 'antibiotics, vet drugs', 'allergens',
       'allergens', 'abnormal smell', 'allergens', 'allergens',
       'abnormal smell', 'Aflatoxin', 'allergens', 'allergens',
       'abnormal smell', 'abnormal smell', 'abnormal smell', 'allergens',
       'almond', 'antibiotics, vet drugs', 'allergens', 'alcohol content',
       'allergens', 'Aflatoxin', 'Aflatoxin', 'allergens',
       'antibiotics, vet drugs', 'Aflatoxin', 'abnormal smell',
       'abnormal smell', 'abnormal smell', 'Aflatoxin', 'allergens',
       'allergens', 'Aflatoxin', 'allergens', 'allergens', 'allergens',
       'abnormal smell', 'allergens', 'Aflatoxin', 'allergens',
       'Aflatoxin', 'alcohol content', 'allergens', 'allergens',
       'allergens', 'alcohol content', 'allergens', 'abnormal smell',
       'allergens', 'antibiotics, vet drugs', 'al

In [107]:
predictions_named_hazard.shape[0]


565

### Product - Category Sub Task 2 

In [108]:
# call model classifier for product-category 
product_category_classifier = classifiers['product-category']

# vectorized input data 
vectorizer = vectorizers['product-category']#pass TF-idf of training TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2, 5), max_df=0.5, min_df=5,max_features=2000)
vectorizer.fit_transform(df_augmented['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)
X_val=vectorizer.transform(testset_competition['text']) # for the column "text" as it has better performance X-boost simple (menaing without tuning)

# isolate model and label encoder 
model_product_category_classifier = product_category_classifier['model'] # save trained model X-Boost 
label_encoder_product_category_classifier = product_category_classifier['label_encoder'] # Save trained model label_encoder 

In [109]:
predictions_product_category= model_product_category_classifier.predict(X_val)
predictions_product_category

array([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       15, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13,  1, 13, 13, 13, 13, 13, 13, 13, 13, 13,  9, 13,
       13, 20, 20,  1,  1, 13, 13, 15, 13, 13, 13, 13, 13, 13, 15, 13, 13,
       13, 13, 15, 13, 13, 13, 13, 13,  9, 13, 13, 13, 13, 13, 13, 13, 13,
       13,  1, 13,  1,  1, 13, 13, 13, 13, 18, 13,  1, 13, 13, 13, 13, 13,
       13, 13, 15, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 15, 13, 13, 13, 13, 13, 13, 13, 15,  9, 18, 13, 13, 20, 13,
        1, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       20, 13,  9, 13, 13, 13, 13, 20, 13, 13, 13, 13, 13, 20, 13, 13, 13,
       13, 13, 13,  9, 13, 13, 13, 13,  1, 13, 13, 13, 19, 13, 13, 18, 13,
       13, 13, 13,  1, 13

In [110]:
product_predictions_category_named = label_encoder_product_category_classifier.inverse_transform(predictions_product_category)  # Get original class names        predictions_dict[label] = predictions_named  # Store predictions with names
product_predictions_category_named

array(['meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
       'meat, egg and dairy products', 'meat, egg and dairy products',
      

### Save to csv hazard-product category (sub task 1)

- we will save the predictions for the `sub task 1` in the folder file : `data_augmented_submission/st1_initial`.

In [111]:
# Create folder structure if it doesn't exist
base_dir = "data_augmented_submission/st1_initial"
os.makedirs(base_dir, exist_ok=True)
# Hazard product data
data_st1 = {
    "hazard": predictions_named_hazard,
    "product": product_predictions_category_named
}
df_st1 = pd.DataFrame(data_st1)
# Save to a CSV
filecsv_file_path = os.path.join(base_dir, "submission.csv")
df_st1.to_csv(filecsv_file_path, index=False)

In [113]:
classifiers['product-category'], classifiers['hazard-category'], classifiers['product'], classifiers['hazard']

({'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric='mlogloss',
                feature_types=None, gamma=None, grow_policy=None,
                importance_type=None, interaction_constraints=None,
                learning_rate=0.2, max_bin=None, max_cat_threshold=None,
                max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
                max_leaves=None, min_child_weight=None, missing=nan,
                monotone_constraints=None, multi_strategy=None, n_estimators=50,
                n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...),
  'label_encoder': LabelEncoder()},
 {'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytre

- All indiviadual besst classifiers with X-Boost and Input `text` have the following common hyperparameters values 
    - `max_depth` = `8`
    - `learning rate` = `0.2`
    - `n_estimators` = `50`

### Lifeboard / Comments 

- The scores were extremely close to 0. 
- Unfortunately, we started the whole project with the wrong data.
- We had to strat with the intial data having only a basic nlp preprocess
- And then use teh augmented data and quich make prediction in the unlabeled to understand something goes wrong because teh reports were misleading . . .
- If we assumed taht the augmented data were "corerct" we futher tuned the models or trying tf-idf with max_features = 5000 insatd of 2000 and PCA, too.  

### Grid Searh TF-idf & PCA reduction dimensioanlity 


- here is a prototype code but we cannot re-run due to time limitations.
    - we have `24` fits ... (cross validation and hyeprparameters tuning . . . )

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import os

def train_xgboost_classifiers_tuning_pca(dataframe, feature_column):
    """
    Train XGBoost classifiers for four labels with PCA and hyperparameter tuning using GridSearchCV.

    Args:
        dataframe: The input dataframe containing the dataset.
        feature_column: The name of the column in the dataframe to be used as features.

    Returns:
        classifiers: A dictionary containing trained classifiers for each label.
        vectorizers: A dictionary containing TF-IDF vectorizers for each label.
        custom_metrics: A dictionary containing the custom metric score for each pair of labels on test data.
    """
    np.random.seed(42)  # For reproducibility

    classifiers = {}  # Dictionary to store the trained classifiers
    vectorizers = {}  # Dictionary to store the TF-IDF vectorizers
    custom_metrics = {}  # Dictionary to store custom metric scores

    # Dictionaries to store test data for each category
    test_data = {}

    # Train classifiers for each label
    for label in ('hazard-category', 'product-category', 'hazard', 'product'):
        print(f"Training classifier for label: {label}")

        # Train-test split with stratification based on the current label
        trainset, testset = train_test_split(
            dataframe,
            test_size=0.2,
            random_state=2024,
            stratify=dataframe[label]  # Maintain class distribution
        )

        # Extract train and test features
        X_train = trainset[feature_column]
        X_test = testset[feature_column]

        # Target
        y_train = trainset[label]
        y_test = testset[label]

        # Encode target labels into numeric values
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)

        # Define TfidfVectorizer for the current label
        vectorizer = TfidfVectorizer(
            strip_accents='unicode', analyzer='char', ngram_range=(2, 5),
            max_df=0.5, min_df=5, max_features=5000 # increase max_featurees 
        )
        vectorizers[label] = vectorizer

        # Transform features using the label-specific vectorizer
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Apply PCA for dimensionality reduction
        pca = PCA(n_components=0.95, random_state=2024)  # Retain 95% variance
        X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
        X_test_pca = pca.transform(X_test_tfidf.toarray()) # use PCS reduction of space 

        # Define XGBoost parameters for GridSearch
        param_grid = {
            'max_depth': [6, 8],
            'n_estimators': [50, 100],
            'learning_rate': [0.1, 0.2]
        }

        # Initialize XGBoost classifier
        xgb = XGBClassifier(
            # use_label_encoder=False,
            eval_metric='mlogloss',
            objective='multi:softmax',
            random_state=2024
        )

        # Perform GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(
            estimator=xgb,
            param_grid=param_grid,
            scoring='f1_macro',
            cv=3,
            verbose=1
        )
        grid_search.fit(X_train_pca, y_train_encoded)

        # Store the best estimator
        classifier = grid_search.best_estimator_
        classifiers[label] = {
            'model': classifier,
            'label_encoder': label_encoder
        }

        # Store test data
        test_data[label] = {
            'X_test_pca': X_test_pca,
            'y_test': y_test_encoded
        }

        # Predict and evaluate
        predictions = classifier.predict(X_test_pca)
        xgb_f1 = f1_score(y_test_encoded, predictions, average='macro', zero_division=0)
        print(f"F1 Score for {label}: {xgb_f1:.3f}")

        # Generate classification report
        report = classification_report(y_test_encoded, predictions, zero_division=0, target_names=label_encoder.classes_)
        print(report)

        # Save the report
        os.makedirs('reports/xgboost', exist_ok=True)
        with open(f'reports/xgboost/xgboost_classifier_report_{label}_{feature_column}.txt', 'w') as xgb_file:
            xgb_file.write(f"Classification Report for {label}_{feature_column}:\n")
            xgb_file.write(report)
            xgb_file.write(f"F1 Score: {xgb_f1:.3f}\n")

    # Compute the custom metric for hazards and products using test data only
    custom_metrics['subtask_1'] = compute_score(
        pd.Series(test_data['hazard-category']['y_test']),
        pd.Series(test_data['product-category']['y_test']),
        pd.Series(classifiers['hazard-category']['model'].predict(test_data['hazard-category']['X_test_pca'])),
        pd.Series(classifiers['product-category']['model'].predict(test_data['product-category']['X_test_pca']))
    )

    custom_metrics['subtask_2'] = compute_score(
        pd.Series(test_data['hazard']['y_test']),
        pd.Series(test_data['product']['y_test']),
        pd.Series(classifiers['hazard']['model'].predict(test_data['hazard']['X_test_pca'])),
        pd.Series(classifiers['product']['model'].predict(test_data['product']['X_test_pca']))
    )

    print(f"Custom Metric for Subtask 1 (Test Data): {custom_metrics['subtask_1']:.3f}")
    print(f"Custom Metric for Subtask 2 (Test Data): {custom_metrics['subtask_2']:.3f}")

    return classifiers, vectorizers, custom_metrics


In [137]:
classifiers, vectorizers, custom_metrics = train_xgboost_classifiers_tuning_pca(df_augmented, 'text')

Training classifier for label: hazard-category
Fitting 3 folds for each of 8 candidates, totalling 24 fits
F1 Score for hazard-category: 0.729
                                precision    recall  f1-score   support

                     allergens       0.91      0.97      0.94       863
                    biological       0.90      0.95      0.93       771
                      chemical       0.77      0.89      0.83       329
food additives and flavourings       1.00      0.36      0.53        14
                foreign bodies       0.83      0.77      0.80       253
                         fraud       0.77      0.68      0.72       280
                     migration       1.00      0.97      0.98        29
          organoleptic aspects       0.88      0.18      0.30        39
                  other hazard       0.93      0.55      0.70       103
              packaging defect       0.94      0.41      0.57        39

                      accuracy                           0.87  

KeyboardInterrupt: 